# Decentralized MWEN via OB-ADMM

The following codes correspond to the centralized optimization of the combined micro water-energy nexus as well as the decentralized approaches using the standard and the objective-based ADMM algorithms. This serves as a direct comparison between Standard ADMM and OB-ADMM approaches, backed againts the results of the benchmark centralized approach.

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

## Water Pumps Linearization

In [ ]:
# WATER PUMPS DATA
PUMP1_DATA = pd.read_excel("Grundfos Pumps.xlsx",sheet_name="Pump 1")
PUMP2_DATA = pd.read_excel("Grundfos Pumps.xlsx",sheet_name="Pump 2")
W1 = PUMP1_DATA.iloc[2:16,1].values # Water flow rate datapoints
Y1 = PUMP1_DATA.iloc[2:16,7].values # Electrical power input datapoints
W2 = PUMP2_DATA.iloc[2:14,1].values # Water flow rate datapoints
Y2 = PUMP2_DATA.iloc[2:14,7].values # Electrical power input datapoints

# Function set size control variables
v_f = 4 # Number of piecewise linear functions
BigM = 1e3 # A large number used for logic statements within mixed-integer programming

In [ ]:
# PUMP 1 PIECEWISE LINEARIZATION
m = Y1.size # Number of datapoints

opt_PWL1 = SolverFactory('gurobi') # Constructing optimization model for piecewise linearization
model_PWL1 = ConcreteModel()

# Sets
model_PWL1.M = RangeSet(1, m) # Datapoints set
model_PWL1.V = RangeSet(1, v_f) # PWL functions set
model_PWL1.V_minus1 = RangeSet(1, v_f-1) # Set of approximated values

# Binary Variables
model_PWL1.alpha = Var(model_PWL1.V_minus1, model_PWL1.M, domain=Binary) # Binary activation variable for piecewise linear segments
# Continuous Variables
model_PWL1.P_pump = Var(model_PWL1.V_minus1, model_PWL1.M) # Approximated pump power input
model_PWL1.a = Var(model_PWL1.V) # Coefficient 1
model_PWL1.b = Var(model_PWL1.V) # Coefficient 2

# Objective Function
def obj_fxn(model_PWL1):  # Eq. (29)
    f = 0
    for i in model_PWL1.M:
        f += (model_PWL1.P_pump[v_f-1,i] - Y1[i-1])**2 # Minimizing square difference between obtained function and actual datapoint
    return f
model_PWL1.OBJ = Objective(rule=obj_fxn, sense=minimize)

# Constraints
def Constraint1_lower_rule(model_PWL1, i): # Eq. (30)
    return model_PWL1.a[1]*W1[i-1] + model_PWL1.b[1] <= model_PWL1.P_pump[1,i]
model_PWL1.Constraint1_lower = Constraint(model_PWL1.M, rule=Constraint1_lower_rule)
def Constraint1_upper_rule(model_PWL1, i):
    return model_PWL1.P_pump[1,i] <= model_PWL1.a[1]*W1[i-1] + model_PWL1.b[1] + model_PWL1.alpha[1,i]*BigM
model_PWL1.Constraint1_upper = Constraint(model_PWL1.M, rule=Constraint1_upper_rule)

def Constraint2_lower_rule(model_PWL1, i): # Eq. (31)
    return model_PWL1.a[2]*W1[i-1] + model_PWL1.b[2] <= model_PWL1.P_pump[1,i]
model_PWL1.Constraint2_lower = Constraint(model_PWL1.M, rule=Constraint2_lower_rule)
def Constraint2_upper_rule(model_PWL1, i):
    return model_PWL1.P_pump[1,i] <= model_PWL1.a[2]*W1[i-1] + model_PWL1.b[2] + (1 - model_PWL1.alpha[1,i])*BigM
model_PWL1.Constraint2_upper = Constraint(model_PWL1.M, rule=Constraint2_upper_rule)

def Constraint3_lower_rule(model_PWL1, i, v): # Eq. (32)
    if v >= 3:
        return model_PWL1.P_pump[v-2,i] <= model_PWL1.P_pump[v-1,i]
    else:
        return Constraint.Skip
model_PWL1.Constraint3_lower = Constraint(model_PWL1.M, model_PWL1.V, rule=Constraint3_lower_rule)
def Constraint3_upper_rule(model_PWL1, i, v):
    if v >= 3:
        return model_PWL1.P_pump[v-1,i] <= model_PWL1.P_pump[v-2,i] + model_PWL1.alpha[v-1,i]*BigM
    else:
        return Constraint.Skip
model_PWL1.Constraint3_upper = Constraint(model_PWL1.M, model_PWL1.V, rule=Constraint3_upper_rule)

def Constraint4_lower_rule(model_PWL1, i, v): # Eq. (33)
    if v >= 3:
        return model_PWL1.a[v]*W1[i-1] + model_PWL1.b[v] <= model_PWL1.P_pump[v-1,i]
    else:
        return Constraint.Skip
model_PWL1.Constraint4_lower = Constraint(model_PWL1.M, model_PWL1.V, rule=Constraint4_lower_rule)
def Constraint4_upper_rule(model_PWL1, i, v):
    if v >= 3:
        return model_PWL1.P_pump[v-1,i] <= model_PWL1.a[v]*W1[i-1] + model_PWL1.b[v] + (1 - model_PWL1.alpha[v-1,i])*BigM
    else:
        return Constraint.Skip
model_PWL1.Constraint4_upper = Constraint(model_PWL1.M, model_PWL1.V, rule=Constraint4_upper_rule)

# PWL RESULTS FOR PUMP 1
Results_PWL1 = opt_PWL1.solve(model_PWL1) # Executing and solving optimization
if Results_PWL1.Problem.Upper_bound == 0 and Results_PWL1.Problem.Lower_bound == 0:
    Optimality_Gap = 0
elif Results_PWL1.Problem.Upper_bound == 0:
    Optimality_Gap = 1e16 # Infinite MIP Gap
else:
    Optimality_Gap = abs(Results_PWL1.Problem.Upper_bound - Results_PWL1.Problem.Lower_bound) / abs(Results_PWL1.Problem.Upper_bound)
    
# Obtaining Coefficients
a_1 = np.zeros(v_f)
b_1 = np.zeros(v_f)
for v in model_PWL1.V:
    a_1[v-1] = value(model_PWL1.a[v])
    b_1[v-1] = value(model_PWL1.b[v])

print(f"Piecewise Linearization Pump 1 Complete.\nMIP Gap: {Optimality_Gap} .")

In [ ]:
# PUMP 2 PIECEWISE LINEARIZATION
m = Y2.size # Number of datapoints

opt_PWL2 = SolverFactory('gurobi') # Constructing optimization model for piecewise linearization
model_PWL2 = ConcreteModel()

# Sets
model_PWL2.M = RangeSet(1, m) # Datapoints set
model_PWL2.V = RangeSet(1, v_f) # PWL functions set
model_PWL2.V_minus1 = RangeSet(1, v_f-1) # Set of approximated values

# Binary Variables
model_PWL2.alpha = Var(model_PWL2.V_minus1, model_PWL2.M, domain=Binary) # Binary activation variable for piecewise linear segments
# Continuous Variables
model_PWL2.P_pump = Var(model_PWL2.V_minus1, model_PWL2.M) # Approximated pump power input
model_PWL2.a = Var(model_PWL2.V) # Coefficient 1
model_PWL2.b = Var(model_PWL2.V) # Coefficient 2

# Objective Function
def obj_fxn(model_PWL2): # Eq. (29)
    f = 0
    for i in model_PWL2.M:
        f += (model_PWL2.P_pump[v_f-1,i] - Y2[i-1])**2 # Minimizing square difference between obtained function and actual datapoint
    return f
model_PWL2.OBJ = Objective(rule=obj_fxn, sense=minimize)

# Constraints
def Constraint1_lower_rule(model_PWL2, i): # Eq. (30)
    return model_PWL2.a[1]*W2[i-1] + model_PWL2.b[1] <= model_PWL2.P_pump[1,i]
model_PWL2.Constraint1_lower = Constraint(model_PWL2.M, rule=Constraint1_lower_rule)
def Constraint1_upper_rule(model_PWL2, i):
    return model_PWL2.P_pump[1,i] <= model_PWL2.a[1]*W2[i-1] + model_PWL2.b[1] + model_PWL2.alpha[1,i]*BigM
model_PWL2.Constraint1_upper = Constraint(model_PWL2.M, rule=Constraint1_upper_rule)

def Constraint2_lower_rule(model_PWL2, i): # Eq. (31)
    return model_PWL2.a[2]*W2[i-1] + model_PWL2.b[2] <= model_PWL2.P_pump[1,i]
model_PWL2.Constraint2_lower = Constraint(model_PWL2.M, rule=Constraint2_lower_rule)
def Constraint2_upper_rule(model_PWL2, i):
    return model_PWL2.P_pump[1,i] <= model_PWL2.a[2]*W2[i-1] + model_PWL2.b[2] + (1 - model_PWL2.alpha[1,i])*BigM
model_PWL2.Constraint2_upper = Constraint(model_PWL2.M, rule=Constraint2_upper_rule)

def Constraint3_lower_rule(model_PWL2, i, v): # Eq. (32)
    if v >= 3:
        return model_PWL2.P_pump[v-2,i] <= model_PWL2.P_pump[v-1,i]
    else:
        return Constraint.Skip
model_PWL2.Constraint3_lower = Constraint(model_PWL2.M, model_PWL2.V, rule=Constraint3_lower_rule)
def Constraint3_upper_rule(model_PWL2, i, v):
    if v >= 3:
        return model_PWL2.P_pump[v-1,i] <= model_PWL2.P_pump[v-2,i] + model_PWL2.alpha[v-1,i]*BigM
    else:
        return Constraint.Skip
model_PWL2.Constraint3_upper = Constraint(model_PWL2.M, model_PWL2.V, rule=Constraint3_upper_rule)

def Constraint4_lower_rule(model_PWL2, i, v): # Eq. (33)
    if v >= 3:
        return model_PWL2.a[v]*W2[i-1] + model_PWL2.b[v] <= model_PWL2.P_pump[v-1,i]
    else:
        return Constraint.Skip
model_PWL2.Constraint4_lower = Constraint(model_PWL2.M, model_PWL2.V, rule=Constraint4_lower_rule)
def Constraint4_upper_rule(model_PWL2, i, v):
    if v >= 3:
        return model_PWL2.P_pump[v-1,i] <= model_PWL2.a[v]*W2[i-1] + model_PWL2.b[v] + (1 - model_PWL2.alpha[v-1,i])*BigM
    else:
        return Constraint.Skip
model_PWL2.Constraint4_upper = Constraint(model_PWL2.M, model_PWL2.V, rule=Constraint4_upper_rule)

# PWL RESULTS FOR PUMP 2
Results_PWL2 = opt_PWL2.solve(model_PWL2) # Executing and solving optimization
if Results_PWL2.Problem.Upper_bound == 0 and Results_PWL2.Problem.Lower_bound == 0:
    Optimality_Gap = 0
elif Results_PWL2.Problem.Upper_bound == 0:
    Optimality_Gap = 1e16 # Infinite MIP Gap
else:
    Optimality_Gap = abs(Results_PWL2.Problem.Upper_bound - Results_PWL2.Problem.Lower_bound) / abs(Results_PWL2.Problem.Upper_bound)
    
# Obtaining Coefficients
a_2 = np.zeros(v_f)
b_2 = np.zeros(v_f)
for v in model_PWL2.V:
    a_2[v-1] = value(model_PWL2.a[v])
    b_2[v-1] = value(model_PWL2.b[v])

print(f"Piecewise Linearization Pump 2 Complete.\nMIP Gap: {Optimality_Gap} .")

## Test Cases

Execute only one Test Case block to load up one of the test cases

In [ ]:
# TEST CASE 1 PARAMETERS
Test_Case = 1 # Indicates which test case has been generated

n_res = 70 # Number of residential units
n_comm = 3 # Number of commercial units
delta_t = 1 # Time step [hrs]

# Power Demand
Pow_Demand = pd.read_excel("Demand Profiles.xlsx",sheet_name="Electricity Demand")
L_res = Pow_Demand.iloc[1:25, 1].values
L_comm = Pow_Demand.iloc[29:53,1].values
P_LOAD = n_res*L_res + n_comm*L_comm # [kW]

# Solar Power
Solar_Profile = pd.read_excel("Demand Profiles.xlsx",sheet_name="Solar Power")
PV_output = Solar_Profile.iloc[1:25,1].values # [kW]
P_SOLAR = n_res*PV_output + 3*n_comm*PV_output # [kW]

# Net Load
P_net = P_LOAD - P_SOLAR # [kW]

# Generators
P_G_min = np.array([40]) # [kW]
P_G_max = np.array([400]) # [kW]
C_G = np.array([0.04]) # [$/kWh]
NL_G = np.array([6.00]) # [$/h]

# Energy Storage
P_ES_lim = np.array([550]) # [kW]
EL_ES_max = np.array([2750]) # [kWh]
EL_ES_min = 0.1*EL_ES_max # [kWh]
EL_ES_init = EL_ES_min # [kWh]
eff_ESc = np.array([0.92])
eff_ESd = np.array([0.95])

# Main Grid
Grid_Price = pd.read_excel("Demand Profiles.xlsx",sheet_name="Grid Prices")
C_grid_plus = Grid_Price.iloc[1:25,1].values # [$/kWh]
C_grid_minus = Grid_Price.iloc[1:25,2].values # [$/kWh]
P_grid_lim = 1200 # [kW]

m3_to_gal = 264.172
# Water Demand
Water_Demand = pd.read_excel("Demand Profiles.xlsx",sheet_name="Water Demand")
D_res_1 = Water_Demand.iloc[1:25,2].values # [gal/h]
D_res_2 = Water_Demand.iloc[1:25,7].values # [gal/h]
D_comm = Water_Demand.iloc[30:54,2].values # [gal/h]

W_LOAD = 0.8*n_res*D_res_1 + 0.2*D_res_2 + n_comm*D_comm # [gal/h]

# Wastewater
w_r = 0.6 # Reclaimed wastewater ratio
W_WW_min = 180 # [gal/h]
W_WW_max = 720 # [gal/h]
WL_WW_lim = 10000 # [gal]
WL_WW_init = 0.05*WL_WW_lim # [gal]
C_WW = 52/m3_to_gal # [kWh/gal]
WR_WW = np.zeros(W_LOAD.size)
WR_WW[0] = w_r*W_LOAD[-1]
for t in range(1,W_LOAD.size):
    WR_WW[t] = w_r*W_LOAD[t-1]

# Regular Water Treatment
W_WT_min = np.array([180]) # [gal/h]
W_WT_max = np.array([720]) # [gal/h]
C_WT = np.array([0.154/m3_to_gal]) # [kWh/gal]

# Water Storage
W_ST_min = np.array([180]) # [gal/h]
W_ST_max = np.array([720]) # [gal/h]
WL_ST_max = np.array([7200]) # [gal]
WL_ST_min = 0.05*WL_ST_max # [gal]
WL_ST_init = WL_ST_min # [gal]
WL_loss = 0.01 # Percent of water loss on water storage system

# Main Municipal Water Network
W_main_lim = 900 # [gal/h]
C_W_main = 0.01 # [$/gal]

# Water Pumps
a_WW = a_1 # [kWh/gal]
b_WW = b_1 # [kWh]

a_WT = np.array([a_1]) # [kWh/gal]
b_WT = np.array([b_1]) # [kWh]
a_WT = np.transpose(a_WT)
b_WT = np.transpose(b_WT)

a_ST = np.array([a_2]) # [kWh/gal]
b_ST = np.array([b_2]) # [kWh]
a_ST = np.transpose(a_ST)
b_ST = np.transpose(b_ST)

In [ ]:
# TEST CASE 2 PARAMETERS
Test_Case = 2 # Indicates which test case has been generated

n_res = 100 # Number of residential units
n_comm = 4 # Number of commercial units
delta_t = 1 # Time step [hrs]

# Power Demand
Pow_Demand = pd.read_excel("Demand Profiles.xlsx",sheet_name="Electricity Demand")
L_res = Pow_Demand.iloc[1:25,2].values
L_comm = Pow_Demand.iloc[29:53,1].values
P_LOAD = n_res*L_res + n_comm*L_comm # [kW]

# Solar Power
Solar_Profile = pd.read_excel("Demand Profiles.xlsx",sheet_name="Solar Power")
PV_output = Solar_Profile.iloc[1:25,2].values # [kW]
P_SOLAR = n_res*PV_output + 3*n_comm*PV_output # [kW]

# Net Load
P_net = P_LOAD - P_SOLAR # [kW]

# Generators
P_G_min = np.array([50]) # [kW]
P_G_max = np.array([500]) # [kW]
C_G = np.array([0.24]) # [$/kWh]
NL_G = np.array([5.00]) # [$/h]

# Energy Storage
P_ES_lim = np.array([550]) # [kW]
EL_ES_max = np.array([2750]) # [kWh]
EL_ES_min = 0.1*EL_ES_max # [kWh]
EL_ES_init = EL_ES_min # [kWh]
eff_ESc = np.array([0.92])
eff_ESd = np.array([0.95])

# Main Grid
Grid_Price = pd.read_excel("Demand Profiles.xlsx",sheet_name="Grid Prices")
C_grid_plus = Grid_Price.iloc[1:25,1].values # [$/kWh]
C_grid_minus = Grid_Price.iloc[1:25,2].values # [$/kWh]
P_grid_lim = 1400 # [kW]

m3_to_gal = 264.172
# Water Demand
Water_Demand = pd.read_excel("Demand Profiles.xlsx",sheet_name="Water Demand")
D_res_1 = Water_Demand.iloc[1:25,2].values # [gal/h]
D_res_2 = Water_Demand.iloc[1:25,7].values # [gal/h]
D_comm = Water_Demand.iloc[30:54,2].values # [gal/h]

W_LOAD = 0.6*n_res*D_res_1 + 0.4*D_res_2 + n_comm*D_comm # [gal/h]

# Wastewater
w_r = 0.5 # Reclaimed wastewater ratio
W_WW_min = 180 # [gal/h]
W_WW_max = 720 # [gal/h]
WL_WW_lim = 10000 # [gal]
WL_WW_init = 0.05*WL_WW_lim # [gal]
C_WW = 52/m3_to_gal # [kWh/gal]
WR_WW = np.zeros(W_LOAD.size)
WR_WW[0] = w_r*W_LOAD[-1]
for t in range(1,W_LOAD.size):
    WR_WW[t] = w_r*W_LOAD[t-1]

# Regular Water Treatment
W_WT_min = np.array([180]) # [gal/h]
W_WT_max = np.array([720]) # [gal/h]
C_WT = np.array([3.6/m3_to_gal]) # [kWh/gal]

# Water Storage
W_ST_min = np.array([180]) # [gal/h]
W_ST_max = np.array([720]) # [gal/h]
WL_ST_max = np.array([7200]) # [gal]
WL_ST_min = 0.05*WL_ST_max # [gal]
WL_ST_init = WL_ST_min # [gal]
WL_loss = 0.01 # Percent of water loss on water storage system

# Main Municipal Water Network
W_main_lim = 900 # [gal/h]
C_W_main = 0.01 # [$/gal]

# Water Pumps
a_WW = a_1 # [kWh/gal]
b_WW = b_1 # [kWh]

a_WT = np.array([a_2]) # [kWh/gal]
b_WT = np.array([b_2]) # [kWh]
a_WT = np.transpose(a_WT)
b_WT = np.transpose(b_WT)

a_ST = np.array([a_2]) # [kWh/gal]
b_ST = np.array([b_2]) # [kWh]
a_ST = np.transpose(a_ST)
b_ST = np.transpose(b_ST)

In [ ]:
# TEST CASE 3 PARAMETERS
Test_Case = 3 # Indicates which test case has been generated

n_res = 60 # Number of residential units
n_comm = 2 # Number of commercial units
delta_t = 1 # Time step [hrs]

# Power Demand
Pow_Demand = pd.read_excel("Demand Profiles.xlsx",sheet_name="Electricity Demand")
L_res = Pow_Demand.iloc[1:25,2].values
L_comm = Pow_Demand.iloc[29:53,1].values
P_LOAD = n_res*L_res + n_comm*L_comm # [kW]

# Solar Power
Solar_Profile = pd.read_excel("Demand Profiles.xlsx",sheet_name="Solar Power")
PV_output = Solar_Profile.iloc[1:25,2].values # [kW]
P_SOLAR = n_res*PV_output + 3*n_comm*PV_output # [kW]

# Net Load
P_net = P_LOAD - P_SOLAR # [kW]

# Generators
P_G_min = np.array([40, 50]) # [kW]
P_G_max = np.array([400, 500]) # [kW]
C_G = np.array([0.04, 0.24]) # [$/kWh]
NL_G = np.array([6.00, 5.00]) # [$/h]

# Energy Storage
P_ES_lim = np.array([550]) # [kW]
EL_ES_max = np.array([2750]) # [kWh]
EL_ES_min = 0.1*EL_ES_max # [kWh]
EL_ES_init = EL_ES_min # [kWh]
eff_ESc = np.array([0.92])
eff_ESd = np.array([0.95])

# Main Grid
Grid_Price = pd.read_excel("Demand Profiles.xlsx",sheet_name="Grid Prices")
C_grid_plus = Grid_Price.iloc[1:25,1].values # [$/kWh]
C_grid_minus = Grid_Price.iloc[1:25,2].values # [$/kWh]
P_grid_lim = 0 # [kW]

m3_to_gal = 264.172
# Water Demand
Water_Demand = pd.read_excel("Demand Profiles.xlsx",sheet_name="Water Demand")
D_res_1 = Water_Demand.iloc[1:25,2].values # [gal/h]
D_res_2 = Water_Demand.iloc[1:25,7].values # [gal/h]
D_comm = Water_Demand.iloc[30:54,2].values # [gal/h]

W_LOAD = 0.7*n_res*D_res_1 + 0.3*D_res_2 + n_comm*D_comm # [gal/h]

# Wastewater
w_r = 0.5 # Reclaimed wastewater ratio
W_WW_min = 180 # [gal/h]
W_WW_max = 720 # [gal/h]
WL_WW_lim = 10000 # [gal]
WL_WW_init = 0.05*WL_WW_lim # [gal]
C_WW = 52/m3_to_gal # [kWh/gal]
WR_WW = np.zeros(W_LOAD.size)
WR_WW[0] = w_r*W_LOAD[-1]
for t in range(1,W_LOAD.size):
    WR_WW[t] = w_r*W_LOAD[t-1]

# Regular Water Treatment
W_WT_min = np.array([180]) # [gal/h]
W_WT_max = np.array([720]) # [gal/h]
C_WT = np.array([3.6/m3_to_gal]) # [kWh/gal]

# Water Storage
W_ST_min = np.array([180]) # [gal/h]
W_ST_max = np.array([720]) # [gal/h]
WL_ST_max = np.array([7200]) # [gal]
WL_ST_min = 0.05*WL_ST_max # [gal]
WL_ST_init = WL_ST_min # [gal]
WL_loss = 0.01 # Percent of water loss on water storage system

# Main Municipal Water Network
W_main_lim = 0 # [gal/h]
C_W_main = 0.01 # [$/gal]

# Water Pumps
a_WW = a_1 # [kWh/gal]
b_WW = b_1 # [kWh]

a_WT = np.array([a_2]) # [kWh/gal]
b_WT = np.array([b_2]) # [kWh]
a_WT = np.transpose(a_WT)
b_WT = np.transpose(b_WT)

a_ST = np.array([a_2]) # [kWh/gal]
b_ST = np.array([b_2]) # [kWh]
a_ST = np.transpose(a_ST)
b_ST = np.transpose(b_ST)

## MWEN Centralized Co-Optimization

In [ ]:
# CENTRALIZED MWEN CO-OPTIMIZATION MODEL
opt_CMWEN = SolverFactory('gurobi') # Constructing optimization model
model_CMWEN = ConcreteModel()

# Sets
model_CMWEN.G = RangeSet(1, C_G.size) # Generators set
model_CMWEN.ES = RangeSet(1, P_ES_lim.size) # Energy storage units set
model_CMWEN.WT = RangeSet(1, W_WT_max.size) # Regular water treatment units set
model_CMWEN.ST = RangeSet(1, WL_ST_max.size) # Water storage tanks set
model_CMWEN.T = RangeSet(1, P_LOAD.size) # Time intervals set
model_CMWEN.V = RangeSet(1, v_f) # PWL functions set
model_CMWEN.V_minus1 = RangeSet(1, v_f-1) # Set of pump power PWL approximated values

# Binary Variables
model_CMWEN.u_G = Var(model_CMWEN.G, model_CMWEN.T, domain=Binary) # Generator status indicatior
model_CMWEN.u_WW = Var(model_CMWEN.T, domain=Binary) # Wastewater plant status indicator
model_CMWEN.u_WT = Var(model_CMWEN.WT, model_CMWEN.T, domain=Binary) # Regular treatment plant status indicator
model_CMWEN.e_ESc = Var(model_CMWEN.ES, model_CMWEN.T, domain=Binary) # Energy storage charging status indicator
model_CMWEN.e_ESd = Var(model_CMWEN.ES, model_CMWEN.T, domain=Binary) # Energy storage discharging status indicator
model_CMWEN.p_plus = Var(model_CMWEN.T, domain=Binary) # Main grid power import indicator
model_CMWEN.p_minus = Var(model_CMWEN.T, domain=Binary) # Main grid power export indicator
model_CMWEN.s_STc = Var(model_CMWEN.ST, model_CMWEN.T, domain=Binary) # Water storage tank fill pump status indicator
model_CMWEN.s_STd = Var(model_CMWEN.ST, model_CMWEN.T, domain=Binary) # Water storage tank release valve status indicator
model_CMWEN.alpha_WW = Var(model_CMWEN.V_minus1, model_CMWEN.T, domain=Binary) # Wastewater Treatment Pump PWL control variable
model_CMWEN.alpha_WT = Var(model_CMWEN.V_minus1, model_CMWEN.WT, model_CMWEN.T, domain=Binary) # WRegular Water Treatment Pump PWL control variable
model_CMWEN.alpha_ST = Var(model_CMWEN.V_minus1, model_CMWEN.ST, model_CMWEN.T, domain=Binary) # Water Storage Pump PWL control variable
# Continuous Variables
model_CMWEN.P_G = Var(model_CMWEN.G, model_CMWEN.T, domain=NonNegativeReals) # Generator power output [kW] 
model_CMWEN.P_ESc = Var(model_CMWEN.ES, model_CMWEN.T, domain=NonNegativeReals) # Energy storage charging input [kW]
model_CMWEN.P_ESd = Var(model_CMWEN.ES, model_CMWEN.T, domain=NonNegativeReals) # Energy storage discharging output [kW]
model_CMWEN.EL_ES = Var(model_CMWEN.ES, model_CMWEN.T, domain=NonNegativeReals) # Energy storage charge level [kWh]
model_CMWEN.P_grid_plus = Var(model_CMWEN.T, domain=NonNegativeReals) # Main grid power import [kW]
model_CMWEN.P_grid_minus = Var(model_CMWEN.T, domain=NonNegativeReals) # Main grid power export [kW]
model_CMWEN.W_WW = Var(model_CMWEN.T, domain=NonNegativeReals) # Wastewater plant water flow rate output [gal/h]
model_CMWEN.WL_WW = Var(model_CMWEN.T, domain=NonNegativeReals) # Wastewater reservoir level [gal]
model_CMWEN.P_WW = Var(model_CMWEN.T, domain=NonNegativeReals) # Wastewater plant power consumption [kW]
model_CMWEN.W_WT = Var(model_CMWEN.WT, model_CMWEN.T, domain=NonNegativeReals) # Regular water treatment plant water flow rate output [gal/h]
model_CMWEN.P_WT = Var(model_CMWEN.WT, model_CMWEN.T, domain=NonNegativeReals) # Regular water treatment plant power consumption [kW]
model_CMWEN.W_STc = Var(model_CMWEN.ST, model_CMWEN.T, domain=NonNegativeReals) # Water storage fill flow rate input [gal/h]
model_CMWEN.W_STd = Var(model_CMWEN.ST, model_CMWEN.T, domain=NonNegativeReals) # Water storage release flow rate output [gal/h]
model_CMWEN.WL_ST = Var(model_CMWEN.ST, model_CMWEN.T, domain=NonNegativeReals) # Water storage tank level [gal]
model_CMWEN.P_WW_pump = Var(model_CMWEN.V_minus1, model_CMWEN.T, domain=NonNegativeReals) # Wastewater pump power consumption [kW]
model_CMWEN.P_WT_pump = Var(model_CMWEN.V_minus1, model_CMWEN.WT, model_CMWEN.T, domain=NonNegativeReals) # Regular water treatment pump power consumption [kW]
model_CMWEN.P_ST_pump = Var(model_CMWEN.V_minus1, model_CMWEN.ST, model_CMWEN.T, domain=NonNegativeReals) # Water storage tank pump power consumption [kW]
model_CMWEN.P_water = Var(model_CMWEN.T, domain=NonNegativeReals) # Total power consumed by water-related components

# Objective Function
def obj_fxn(model_CMWEN): # Eq. (34)
    f_cost = 0
    for t in model_CMWEN.T:
        for g in model_CMWEN.G:
            f_cost += delta_t*(NL_G[g-1]*model_CMWEN.u_G[g,t] + C_G[g-1]*model_CMWEN.P_G[g,t])
        f_cost += C_grid_plus[t-1]*model_CMWEN.P_grid_plus[t] - C_grid_minus[t-1]*model_CMWEN.P_grid_minus[t] # Minimizing combined operation costs
    return f_cost
model_CMWEN.OBJ = Objective(rule=obj_fxn, sense=minimize)

# Constraints
# Generators
def Gen_min_output_rule(model_CMWEN, g, t):
    return P_G_min[g-1]*model_CMWEN.u_G[g,t] <= model_CMWEN.P_G[g,t]
model_CMWEN.Gen_min_output = Constraint(model_CMWEN.G, model_CMWEN.T, rule=Gen_min_output_rule)
def Gen_max_output_rule(model_CMWEN, g, t):
    return model_CMWEN.P_G[g,t] <= P_G_max[g-1]*model_CMWEN.u_G[g,t]
model_CMWEN.Gen_max_output = Constraint(model_CMWEN.G, model_CMWEN.T, rule=Gen_max_output_rule)
# Energy Storage
def ES_charge_limit_rule(model_CMWEN, b, t):
    return model_CMWEN.P_ESc[b,t] <= P_ES_lim[b-1]*model_CMWEN.e_ESc[b,t]
model_CMWEN.ES_charge_limit = Constraint(model_CMWEN.ES, model_CMWEN.T, rule=ES_charge_limit_rule)
def ES_discharge_limit_rule(model_CMWEN, b, t):
    return model_CMWEN.P_ESd[b,t] <= P_ES_lim[b-1]*model_CMWEN.e_ESd[b,t]
model_CMWEN.ES_discharge_limit = Constraint(model_CMWEN.ES, model_CMWEN.T, rule=ES_discharge_limit_rule)
def ES_status_rule(model_CMWEN, b, t):
    return model_CMWEN.e_ESc[b,t] + model_CMWEN.e_ESd[b,t] <= 1
model_CMWEN.ES_status = Constraint(model_CMWEN.ES, model_CMWEN.T, rule=ES_status_rule)
def ES_charge_lvl_rule(model_CMWEN, b, t):
    if t == 1:
        return model_CMWEN.EL_ES[b,t] == EL_ES_init[b-1] + delta_t*(eff_ESc[b-1]*model_CMWEN.P_ESc[b,t] - model_CMWEN.P_ESd[b,t]/eff_ESd[b-1])
    else:
        return model_CMWEN.EL_ES[b,t] == model_CMWEN.EL_ES[b,t-1] + delta_t*(eff_ESc[b-1]*model_CMWEN.P_ESc[b,t] - model_CMWEN.P_ESd[b,t]/eff_ESd[b-1])
model_CMWEN.ES_charge_lvl = Constraint(model_CMWEN.ES, model_CMWEN.T, rule=ES_charge_lvl_rule)
def ES_min_charge_lvl_rule(model_CMWEN, b , t):
    return EL_ES_min[b-1] <= model_CMWEN.EL_ES[b,t]
model_CMWEN.ES_min_charge_lvl = Constraint(model_CMWEN.ES, model_CMWEN.T, rule=ES_min_charge_lvl_rule)
def ES_max_charge_lvl_rule(model_CMWEN, b , t):
    return model_CMWEN.EL_ES[b,t] <= EL_ES_max[b-1]
model_CMWEN.ES_max_charge_lvl = Constraint(model_CMWEN.ES, model_CMWEN.T, rule=ES_max_charge_lvl_rule)
# Main Grid
def Grid_import_limit_rule(model_CMWEN, t):
    return model_CMWEN.P_grid_plus[t] <= P_grid_lim*model_CMWEN.p_plus[t]
model_CMWEN.Grid_import_limit = Constraint(model_CMWEN.T, rule=Grid_import_limit_rule)
def Grid_export_limit_rule(model_CMWEN, t):
    return model_CMWEN.P_grid_minus[t] <= P_grid_lim*model_CMWEN.p_minus[t]
model_CMWEN.Grid_export_limit = Constraint(model_CMWEN.T, rule=Grid_export_limit_rule)
def Grid_status_rule(model_CMWEN, t):
    return model_CMWEN.p_plus[t] + model_CMWEN.p_minus[t] <= 1
model_CMWEN.Grid_status = Constraint(model_CMWEN.T, rule=Grid_status_rule)
# Power Balance
def Power_Balance_rule(model_CMWEN, t):
    return sum(model_CMWEN.P_G[g,t] for g in model_CMWEN.G) + sum(model_CMWEN.P_ESd[b,t] - model_CMWEN.P_ESc[b,t] for b in model_CMWEN.ES) + model_CMWEN.P_grid_plus[t] - model_CMWEN.P_grid_minus[t] == P_LOAD[t-1] - P_SOLAR[t-1] + model_CMWEN.P_water[t]
model_CMWEN.Power_Balance = Constraint(model_CMWEN.T, rule=Power_Balance_rule)
def PowerConsumption_Water_rule(model_CMWEN, t):
    return model_CMWEN.P_water[t] == model_CMWEN.P_WW[t] + model_CMWEN.P_WW_pump[v_f-1,t] + sum(model_CMWEN.P_WT[w,t] + model_CMWEN.P_WT_pump[v_f-1,w,t] for w in model_CMWEN.WT) + sum(model_CMWEN.P_ST_pump[v_f-1,k,t] for k in model_CMWEN.ST)
model_CMWEN.PowerConsumption_Water = Constraint(model_CMWEN.T, rule=PowerConsumption_Water_rule)
# Wastewater
def Wastewater_min_output_rule(model_CMWEN, t):
    return W_WW_min*model_CMWEN.u_WW[t] <= model_CMWEN.W_WW[t]
model_CMWEN.Wastewater_min_output = Constraint(model_CMWEN.T, rule=Wastewater_min_output_rule)
def Wastewater_max_output_rule(model_CMWEN, t):
    return model_CMWEN.W_WW[t] <= W_WW_max*model_CMWEN.u_WW[t]
model_CMWEN.Wastewater_max_output = Constraint(model_CMWEN.T, rule=Wastewater_max_output_rule)
def Wastewater_reservoir_lvl_rule(model_CMWEN, t):
    if t == 1:
        return model_CMWEN.WL_WW[t] == WL_WW_init + delta_t*(WR_WW[t-1] - model_CMWEN.W_WW[t])
    else:
        return model_CMWEN.WL_WW[t] == model_CMWEN.WL_WW[t-1] + delta_t*(WR_WW[t-1] - model_CMWEN.W_WW[t])
model_CMWEN.Wastewater_reservoir_lvl = Constraint(model_CMWEN.T, rule=Wastewater_reservoir_lvl_rule)
def Wastewater_reservoir_limit_rule(model_CMWEN, t):
    return model_CMWEN.WL_WW[t] <= WL_WW_lim
model_CMWEN.Wastewater_reservoir_limit = Constraint(model_CMWEN.T, rule=Wastewater_reservoir_limit_rule)
def Wastewater_PowerConsumption_rule(model_CMWEN, t):
    return model_CMWEN.P_WW[t] == C_WW*model_CMWEN.W_WW[t]
model_CMWEN.Wastewater_PowerConsumption = Constraint(model_CMWEN.T, rule=Wastewater_PowerConsumption_rule)
def Wastewater_Use_rule(model_CMWEN, t):
    return model_CMWEN.WL_WW[model_CMWEN.T.last()] <= 0.1*WL_WW_lim
model_CMWEN.Wastewater_Use = Constraint(model_CMWEN.T, rule=Wastewater_Use_rule) # Restrict end-of-day wastewater level to 10% of its total capacity
# Regular Treatment
def RegWater_min_output_rule(model_CMWEN, w, t):
    return W_WT_min[w-1]*model_CMWEN.u_WT[w,t] <= model_CMWEN.W_WT[w,t]
model_CMWEN.RegWater_min_output = Constraint(model_CMWEN.WT, model_CMWEN.T, rule=RegWater_min_output_rule)
def RegWater_max_output_rule(model_CMWEN, w, t):
    return model_CMWEN.W_WT[w,t] <= W_WT_max[w-1]*model_CMWEN.u_WT[w,t]
model_CMWEN.RegWater_max_output = Constraint(model_CMWEN.WT, model_CMWEN.T, rule=RegWater_max_output_rule)
def RegWater_PowerConsumption_rule(model_CMWEN, w, t):
    return model_CMWEN.P_WT[w,t] == C_WT[w-1]*model_CMWEN.W_WT[w,t]
model_CMWEN.RegWater_PowerConsumption = Constraint(model_CMWEN.WT, model_CMWEN.T, rule=RegWater_PowerConsumption_rule)
# Water Storage
def WaterStorage_min_output_rule(model_CMWEN, k ,t):
    return W_ST_min[k-1]*model_CMWEN.s_STc[k,t] <= model_CMWEN.W_STc[k,t]
model_CMWEN.WaterStorage_min_output = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStorage_min_output_rule)
def WaterStorage_max_output_rule(model_CMWEN, k ,t):
    return model_CMWEN.W_STc[k,t] <= W_ST_max[k-1]*model_CMWEN.s_STc[k,t]
model_CMWEN.WaterStorage_max_output = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStorage_max_output_rule)
def WaterStorage_max_input_rule(model_CMWEN, k ,t):
    return model_CMWEN.W_STd[k,t] <= W_ST_max[k-1]*model_CMWEN.s_STd[k,t]
model_CMWEN.WaterStorage_max_input = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStorage_max_input_rule)
def WaterStorage_status_rule(model_CMWEN, k, t):
    return model_CMWEN.s_STc[k,t] + model_CMWEN.s_STd[k,t] <= 1
model_CMWEN.WaterStorage_status = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStorage_status_rule)
def WaterStorage_Tank_lvl_rule(model_CMWEN, k, t):
    if t == 1:
        return model_CMWEN.WL_ST[k,t] == WL_ST_init[k-1] + delta_t*(model_CMWEN.W_STc[k,t] - model_CMWEN.W_STd[k,t])
    else:
        return model_CMWEN.WL_ST[k,t] == model_CMWEN.WL_ST[k,t-1] + delta_t*(model_CMWEN.W_STc[k,t] - model_CMWEN.W_STd[k,t])
model_CMWEN.WaterStorage_Tank_lvl = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStorage_Tank_lvl_rule)
def WaterStorage_min_Tank_lvl_rule(model_CMWEN, k, t):
    return WL_ST_min[k-1] <= model_CMWEN.WL_ST[k,t]
model_CMWEN.WaterStorage_min_Tank_lvl = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStorage_min_Tank_lvl_rule)
def WaterStorage_max_Tank_lvl_rule(model_CMWEN, k, t):
    return model_CMWEN.WL_ST[k,t] <= WL_ST_max[k-1]
model_CMWEN.WaterStorage_max_Tank_lvl = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStorage_max_Tank_lvl_rule)
# Water Balance
def Water_Balance_rule(model_CMWEN, t):
    return model_CMWEN.W_WW[t] + sum(model_CMWEN.W_WT[w,t] for w in model_CMWEN.WT) + sum(model_CMWEN.W_STd[k,t] - model_CMWEN.W_STc[k,t] for k in model_CMWEN.ST) == W_LOAD[t-1]
model_CMWEN.Water_Balance = Constraint(model_CMWEN.T, rule=Water_Balance_rule)

# Linearized Pump Constraints for each water resource
def WastewaterPump_Constraint1_lower_rule(model_CMWEN, t):
    return a_WW[0]*model_CMWEN.W_WW[t] + b_WW[0]*model_CMWEN.u_WW[t] <= model_CMWEN.P_WW_pump[1,t]
model_CMWEN.WastewaterPump_Constraint1_lower = Constraint(model_CMWEN.T, rule=WastewaterPump_Constraint1_lower_rule)
def WastewaterPump_Constraint1_upper_rule(model_CMWEN, t):
    return model_CMWEN.P_WW_pump[1,t] <= a_WW[0]*model_CMWEN.W_WW[t] + b_WW[0]*model_CMWEN.u_WW[t] + model_CMWEN.alpha_WW[1,t]*BigM
model_CMWEN.WastewaterPump_Constraint1_upper = Constraint(model_CMWEN.T, rule=WastewaterPump_Constraint1_upper_rule)
def WastewaterPump_Constraint2_lower_rule(model_CMWEN, t):
    return a_WW[1]*model_CMWEN.W_WW[t] + b_WW[1]*model_CMWEN.u_WW[t] <= model_CMWEN.P_WW_pump[1,t]
model_CMWEN.WastewaterPump_Constraint2_lower = Constraint(model_CMWEN.T, rule=WastewaterPump_Constraint2_lower_rule)
def WastewaterPump_Constraint2_upper_rule(model_CMWEN, t):
    return model_CMWEN.P_WW_pump[1,t] <= a_WW[1]*model_CMWEN.W_WW[t] + b_WW[1]*model_CMWEN.u_WW[t] + (1 - model_CMWEN.alpha_WW[1,t])*BigM
model_CMWEN.WastewaterPump_Constraint2_upper = Constraint(model_CMWEN.T, rule=WastewaterPump_Constraint2_upper_rule)
def WastewaterPump_Constraint3_lower_rule(model_CMWEN, t, v):
    if v >= 3:
        return model_CMWEN.P_WW_pump[v-2,t] <= model_CMWEN.P_WW_pump[v-1,t]
    else:
        return Constraint.Skip
model_CMWEN.WastewaterPump_Constraint3_lower = Constraint(model_CMWEN.T, model_CMWEN.V, rule=WastewaterPump_Constraint3_lower_rule)
def WastewaterPump_Constraint3_upper_rule(model_CMWEN, t, v):
    if v >= 3:
        return model_CMWEN.P_WW_pump[v-1,t] <= model_CMWEN.P_WW_pump[v-2,t] + model_CMWEN.alpha_WW[v-1,t]*BigM
    else:
        return Constraint.Skip
model_CMWEN.WastewaterPump_Constraint3_upper = Constraint(model_CMWEN.T, model_CMWEN.V, rule=WastewaterPump_Constraint3_upper_rule)
def WastewaterPump_Constraint4_lower_rule(model_CMWEN, t, v):
    if v >= 3:
        return a_WW[v-1]*model_CMWEN.W_WW[t] + b_WW[v-1]*model_CMWEN.u_WW[t] <= model_CMWEN.P_WW_pump[v-1,t]
    else:
        return Constraint.Skip
model_CMWEN.WastewaterPump_Constraint4_lower = Constraint(model_CMWEN.T, model_CMWEN.V, rule=WastewaterPump_Constraint4_lower_rule)
def WastewaterPump_Constraint4_upper_rule(model_CMWEN, t, v):
    if v >= 3:
        return model_CMWEN.P_WW_pump[v-1,t] <= a_WW[v-1]*model_CMWEN.W_WW[t] + b_WW[v-1]*model_CMWEN.u_WW[t] + (1 - model_CMWEN.alpha_WW[v-1,t])*BigM
    else:
        return Constraint.Skip
model_CMWEN.WastewaterPump_Constraint4_upper = Constraint(model_CMWEN.T, model_CMWEN.V, rule=WastewaterPump_Constraint4_upper_rule)

def RegWaterPump_Constraint1_lower_rule(model_CMWEN, w, t):
    return a_WT[0,w-1]*model_CMWEN.W_WT[w,t] + b_WT[0,w-1]*model_CMWEN.u_WT[w,t] <= model_CMWEN.P_WT_pump[1,w,t]
model_CMWEN.RegWaterPump_Constraint1_lower = Constraint(model_CMWEN.WT, model_CMWEN.T, rule=RegWaterPump_Constraint1_lower_rule)
def RegWaterPump_Constraint1_upper_rule(model_CMWEN, w, t):
    return model_CMWEN.P_WT_pump[1,w,t] <= a_WT[0,w-1]*model_CMWEN.W_WT[w,t] + b_WT[0,w-1]*model_CMWEN.u_WT[w,t] + model_CMWEN.alpha_WT[1,w,t]*BigM
model_CMWEN.RegWaterPump_Constraint1_upper = Constraint(model_CMWEN.WT, model_CMWEN.T, rule=RegWaterPump_Constraint1_upper_rule)
def RegWaterPump_Constraint2_lower_rule(model_CMWEN, w, t):
    return a_WT[1,w-1]*model_CMWEN.W_WT[w,t] + b_WT[1,w-1]*model_CMWEN.u_WT[w,t] <= model_CMWEN.P_WT_pump[1,w,t]
model_CMWEN.RegWaterPump_Constraint2_lower = Constraint(model_CMWEN.WT, model_CMWEN.T, rule=RegWaterPump_Constraint2_lower_rule)
def RegWaterPump_Constraint2_upper_rule(model_CMWEN, w, t):
    return model_CMWEN.P_WT_pump[1,w,t] <= a_WT[1,w-1]*model_CMWEN.W_WT[w,t] + b_WT[1,w-1]*model_CMWEN.u_WT[w,t] + (1 - model_CMWEN.alpha_WT[1,w,t])*BigM
model_CMWEN.RegWaterPump_Constraint2_upper = Constraint(model_CMWEN.WT, model_CMWEN.T, rule=RegWaterPump_Constraint2_upper_rule)
def RegWaterPump_Constraint3_lower_rule(model_CMWEN, w, t, v):
    if v >= 3:
        return model_CMWEN.P_WT_pump[v-2,w,t] <= model_CMWEN.P_WT_pump[v-1,w,t]
    else:
        return Constraint.Skip
model_CMWEN.RegWaterPump_Constraint3_lower = Constraint(model_CMWEN.WT, model_CMWEN.T, model_CMWEN.V, rule=RegWaterPump_Constraint3_lower_rule)
def RegWaterPump_Constraint3_upper_rule(model_CMWEN, w, t, v):
    if v >= 3:
        return model_CMWEN.P_WT_pump[v-1,w,t] <= model_CMWEN.P_WT_pump[v-2,w,t] + model_CMWEN.alpha_WT[v-1,w,t]*BigM
    else:
        return Constraint.Skip
model_CMWEN.RegWaterPump_Constraint3_upper = Constraint(model_CMWEN.WT, model_CMWEN.T, model_CMWEN.V, rule=RegWaterPump_Constraint3_upper_rule)
def RegWaterPump_Constraint4_lower_rule(model_CMWEN, w, t, v):
    if v >= 3:
        return a_WT[v-1,w-1]*model_CMWEN.W_WT[w,t] + b_WT[v-1,w-1]*model_CMWEN.u_WT[w,t] <= model_CMWEN.P_WT_pump[v-1,w,t]
    else:
        return Constraint.Skip
model_CMWEN.RegWaterPump_Constraint4_lower = Constraint(model_CMWEN.WT, model_CMWEN.T, model_CMWEN.V, rule=RegWaterPump_Constraint4_lower_rule)
def RegWaterPump_Constraint4_upper_rule(model_CMWEN, w, t, v):
    if v >= 3:
        return model_CMWEN.P_WT_pump[v-1,w,t] <= a_WT[v-1,w-1]*model_CMWEN.W_WT[w,t] + b_WT[v-1,w-1]*model_CMWEN.u_WT[w,t] + (1 - model_CMWEN.alpha_WT[v-1,w,t])*BigM
    else:
        return Constraint.Skip
model_CMWEN.RegWaterPump_Constraint4_upper = Constraint(model_CMWEN.WT, model_CMWEN.T, model_CMWEN.V, rule=RegWaterPump_Constraint4_upper_rule)

def WaterStoragePump_Constraint1_lower_rule(model_CMWEN, k, t):
    return a_ST[0,k-1]*model_CMWEN.W_STc[k,t] + b_ST[0,k-1]*model_CMWEN.s_STc[k,t] <= model_CMWEN.P_ST_pump[1,k,t]
model_CMWEN.WaterStoragePump_Constraint1_lower = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStoragePump_Constraint1_lower_rule)
def WaterStoragePump_Constraint1_upper_rule(model_CMWEN, k, t):
    return model_CMWEN.P_ST_pump[1,k,t] <= a_ST[0,k-1]*model_CMWEN.W_STc[k,t] + b_ST[0,k-1]*model_CMWEN.s_STc[k,t] + model_CMWEN.alpha_ST[1,k,t]*BigM
model_CMWEN.WaterStoragePump_Constraint1_upper = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStoragePump_Constraint1_upper_rule)
def WaterStoragePump_Constraint2_lower_rule(model_CMWEN, k, t):
    return a_ST[1,k-1]*model_CMWEN.W_STc[k,t] + b_ST[1,k-1] <= model_CMWEN.P_ST_pump[1,k,t]
model_CMWEN.WaterStoragePump_Constraint2_lower = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStoragePump_Constraint2_lower_rule)
def WaterStoragePump_Constraint2_upper_rule(model_CMWEN, k, t):
    return model_CMWEN.P_ST_pump[1,k,t] <= a_WT[1,k-1]*model_CMWEN.W_STc[k,t] + b_ST[1,k-1]*model_CMWEN.s_STc[k,t] + (1 - model_CMWEN.alpha_ST[1,k,t])*BigM
model_CMWEN.WaterStoragePump_Constraint2_upper = Constraint(model_CMWEN.ST, model_CMWEN.T, rule=WaterStoragePump_Constraint2_upper_rule)
def WaterStoragePump_Constraint3_lower_rule(model_CMWEN, k, t, v):
    if v >= 3:
        return model_CMWEN.P_ST_pump[v-2,k,t] <= model_CMWEN.P_WT_pump[v-1,k,t]
    else:
        return Constraint.Skip
model_CMWEN.WaterStoragePump_Constraint3_lower = Constraint(model_CMWEN.ST, model_CMWEN.T, model_CMWEN.V, rule=WaterStoragePump_Constraint3_lower_rule)
def WaterStoragePump_Constraint3_upper_rule(model_CMWEN, k, t, v):
    if v >= 3:
        return model_CMWEN.P_ST_pump[v-1,k,t] <= model_CMWEN.P_ST_pump[v-2,k,t] + model_CMWEN.alpha_ST[v-1,k,t]*BigM
    else:
        return Constraint.Skip
model_CMWEN.WaterStoragePump_Constraint3_upper = Constraint(model_CMWEN.ST, model_CMWEN.T, model_CMWEN.V, rule=WaterStoragePump_Constraint3_upper_rule)
def WaterStoragePump_Constraint4_lower_rule(model_CMWEN, k, t, v):
    if v >= 3:
        return a_ST[v-1,k-1]*model_CMWEN.W_STc[k,t] + b_ST[v-1,k-1]*model_CMWEN.s_STc[k,t] <= model_CMWEN.P_ST_pump[v-1,k,t]
    else:
        return Constraint.Skip
model_CMWEN.WaterStoragePump_Constraint4_lower = Constraint(model_CMWEN.ST, model_CMWEN.T, model_CMWEN.V, rule=RegWaterPump_Constraint4_lower_rule)
def WaterStoragePump_Constraint4_upper_rule(model_CMWEN, k, t, v):
    if v >= 3:
        return model_CMWEN.P_ST_pump[v-1,k,t] <= a_ST[v-1,k-1]*model_CMWEN.W_STc[k,t] + b_ST[v-1,k-1]*model_CMWEN.s_STc[k,t] + (1 - model_CMWEN.alpha_ST[v-1,k,t])*BigM
    else:
        return Constraint.Skip
model_CMWEN.WaterStoragePump_Constraint4_upper = Constraint(model_CMWEN.ST, model_CMWEN.T, model_CMWEN.V, rule=WaterStoragePump_Constraint4_upper_rule)

# SOLVING CENTRALIZED MWEN OPTIMIZATION
Results_CMWEN = opt_CMWEN.solve(model_CMWEN)
if Results_CMWEN.Problem.Upper_bound == 0 and Results_CMWEN.Problem.Lower_bound == 0:
    Optimality_Gap_CMWEN = 0
elif Results_CMWEN.Problem.Upper_bound == 0:
    Optimality_Gap_CMWEN = 1e16 # Infinite MIP Gap
else:
    Optimality_Gap_CMWEN = abs(Results_CMWEN.Problem.Upper_bound - Results_CMWEN.Problem.Lower_bound) / abs(Results_CMWEN.Problem.Upper_bound)

print(f"Centralized Micro Water-Energy Nexus (CMWEM) Optimization Complete.\nMIP Gap: {Optimality_Gap_CMWEN} .")

In [ ]:
# Objective Value: Optimal MWEN operation cost for a 24-hr period [$]
print(f'Optimal Cost: ${value(model_CMWEN.OBJ):.2f}')

In [ ]:
# Total MWM Energy Consumption
print('Energy Consumption: {:0.3f} kWh'.format(value(sum(model_CMWEN.P_water[t] for t in model_CMWEN.T))))

In [ ]:
# Total WMW Operation Power Consumption
plt.figure(figsize=(5, 4))
X = np.zeros(W_LOAD.size)
Y = np.zeros(W_LOAD.size)
for t in range(1, W_LOAD.size+1):
    X[t-1] = t
    Y[t-1] = value(model_CMWEN.P_water[t])

plt.bar(X,Y,color='orangered')
plt.xticks(np.arange(2, max(X)+1, 2))
plt.xlabel('Hour')
plt.ylabel('Power [kWh]')
plt.title('Water Management Power Consumption')
plt.grid(True)
plt.gca().set_axisbelow(True)

## Decentralized MWEN Co-Optimization

### Standard ADMM

In [ ]:
# DECENTRALIZED OPTIMIZATION: STANDARD ADMM ALGORITHM
D_start_time = time.time()

# Initializing algorithm variables
P_water_E_SADMM = np.zeros([2, P_LOAD.size]) # Water management power consumption duplicated global variales
P_water_W_SADMM = np.zeros([2, W_LOAD.size])
lam_SADMM = np.zeros([2, P_LOAD.size]) # Lagrange multiplier
r_SADMM = np.ones([2, P_LOAD.size]) # Primal residual
s_SADMM = np.ones([2, P_LOAD.size]) # Dual residual
eps_SADMM = np.ones(2) # Solution feasibility metric
# Individual Objective Value Arrays
OBJ_E_SADMM = np.zeros(2)
OBJ_W_SADMM = np.zeros(2)
OBJ_global_SADMM = np.zeros(2) # Global objective value

rho = 0.01 # Penalty
eps_threshold = 1e-6 # Solution feasibility threshold

k = 0
# ADMM Iteration loop
Check = 1
while Check:
    
    # MEM LOCAL OPTIMIZATION MODEL
    opt_E = SolverFactory('gurobi')
    model_E = ConcreteModel()

    # Sets
    model_E.G = RangeSet(1, C_G.size) # Generators set
    model_E.ES = RangeSet(1, P_ES_lim.size) # Energy storage units set
    model_E.T = RangeSet(1, P_LOAD.size) # Time intervals set

    # Binary Variables
    model_E.u_G = Var(model_E.G, model_E.T, domain=Binary) # Generator status indicatior
    model_E.e_ESc = Var(model_E.ES, model_E.T, domain=Binary) # Energy storage charging status indicator
    model_E.e_ESd = Var(model_E.ES, model_E.T, domain=Binary) # Energy storage discharging status indicator
    model_E.p_plus = Var(model_E.T, domain=Binary) # Main grid power import indicator
    model_E.p_minus = Var(model_E.T, domain=Binary) # Main grid power export indicator
    # Continuous Variables
    model_E.P_G = Var(model_E.G, model_E.T, domain=NonNegativeReals) # Generator power output [kW] 
    model_E.P_ESc = Var(model_E.ES, model_E.T, domain=NonNegativeReals) # Energy storage charging input [kW]
    model_E.P_ESd = Var(model_E.ES, model_E.T, domain=NonNegativeReals) # Energy storage discharging output [kW]
    model_E.EL_ES = Var(model_E.ES, model_E.T, domain=NonNegativeReals) # Energy storage charge level [kWh]
    model_E.P_grid_plus = Var(model_E.T, domain=NonNegativeReals) # Main grid power import [kW]
    model_E.P_grid_minus = Var(model_E.T, domain=NonNegativeReals) # Main grid power export [kW]
    model_E.P_water_E_var = Var(model_E.T, domain=NonNegativeReals) # Power consumption of water components [kW]

    # Objective Function
    def obj_fxn_E(model_E): # Eq. (37)
        f = 0
        for t in model_E.T:
            for g in model_E.G:
                f += delta_t*(NL_G[g-1]*model_E.u_G[g,t] + C_G[g-1]*model_E.P_G[g,t])
            f += C_grid_plus[t-1]*model_E.P_grid_plus[t] - C_grid_minus[t-1]*model_E.P_grid_minus[t]
            f += lam_SADMM[k,t-1]*(model_E.P_water_E_var[t] - P_water_W_SADMM[k,t-1]) + (rho/2)*(model_E.P_water_E_var[t] - P_water_W_SADMM[k,t-1])**2 # Lagrangian augmentation
        return f
    model_E.OBJ = Objective(rule=obj_fxn_E, sense=minimize)

    # Constraints
    # Generators
    def Gen_min_output_rule(model_E, g, t):
        return P_G_min[g-1]*model_E.u_G[g,t] <= model_E.P_G[g,t]
    model_E.Gen_min_output = Constraint(model_E.G, model_E.T, rule=Gen_min_output_rule)
    def Gen_max_output_rule(model_E, g, t):
        return model_E.P_G[g,t] <= P_G_max[g-1]*model_E.u_G[g,t]
    model_E.Gen_max_output = Constraint(model_E.G, model_E.T, rule=Gen_max_output_rule)
    # Energy Storage
    def ES_charge_limit_rule(model_E, b, t):
        return model_E.P_ESc[b,t] <= P_ES_lim[b-1]*model_E.e_ESc[b,t]
    model_E.ES_charge_limit = Constraint(model_E.ES, model_E.T, rule=ES_charge_limit_rule)
    def ES_discharge_limit_rule(model_E, b, t):
        return model_E.P_ESd[b,t] <= P_ES_lim[b-1]*model_E.e_ESd[b,t]
    model_E.ES_discharge_limit = Constraint(model_E.ES, model_E.T, rule=ES_discharge_limit_rule)
    def ES_status_rule(model_E, b, t):
        return model_E.e_ESc[b,t] + model_E.e_ESd[b,t] <= 1
    model_E.ES_status = Constraint(model_E.ES, model_E.T, rule=ES_status_rule)
    def ES_charge_lvl_rule(model_E, b, t):
        if t == 1:
            return model_E.EL_ES[b,t] == EL_ES_init[b-1] + delta_t*(eff_ESc[b-1]*model_E.P_ESc[b,t] - model_E.P_ESd[b,t]/eff_ESd[b-1])
        else:
            return model_E.EL_ES[b,t] == model_E.EL_ES[b,t-1] + delta_t*(eff_ESc[b-1]*model_E.P_ESc[b,t] - model_E.P_ESd[b,t]/eff_ESd[b-1])
    model_E.ES_charge_lvl = Constraint(model_E.ES, model_E.T, rule=ES_charge_lvl_rule)
    def ES_min_charge_lvl_rule(model_E, b , t):
        return EL_ES_min[b-1] <= model_E.EL_ES[b,t]
    model_E.ES_min_charge_lvl = Constraint(model_E.ES, model_E.T, rule=ES_min_charge_lvl_rule)
    def ES_max_charge_lvl_rule(model_E, b , t):
        return model_E.EL_ES[b,t] <= EL_ES_max[b-1]
    model_E.ES_max_charge_lvl = Constraint(model_E.ES, model_E.T, rule=ES_max_charge_lvl_rule)
    # Main Grid
    def Grid_import_limit_rule(model_E, t):
        return model_E.P_grid_plus[t] <= P_grid_lim*model_E.p_plus[t]
    model_E.Grid_import_limit = Constraint(model_E.T, rule=Grid_import_limit_rule)
    def Grid_export_limit_rule(model_E, t):
        return model_E.P_grid_minus[t] <= P_grid_lim*model_E.p_minus[t]
    model_E.Grid_export_limit = Constraint(model_E.T, rule=Grid_export_limit_rule)
    def Grid_status_rule(model_E, t):
        return model_E.p_plus[t] + model_E.p_minus[t] <= 1
    model_E.Grid_status = Constraint(model_E.T, rule=Grid_status_rule)
    # Power Balance
    def Power_Balance_rule(model_E, t):
        return sum(model_E.P_G[g,t] for g in model_E.G) + sum(model_E.P_ESd[b,t] - model_E.P_ESc[b,t] for b in model_E.ES) + model_E.P_grid_plus[t] - model_E.P_grid_minus[t] == P_LOAD[t-1] - P_SOLAR[t-1] + model_E.P_water_E_var[t]
    model_E.Power_Balance = Constraint(model_E.T, rule=Power_Balance_rule)
    
    # MEM Optimization Solution
    Results_E = opt_E.solve(model_E)
    OBJ_E_SADMM[k+1] = value(model_E.OBJ)
    for t in range(P_LOAD.size):
        P_water_E_SADMM[k+1,t] = value(model_E.P_water_E_var[t+1])

        
    # MWM LOCAL OPTIMIZATION MODEL
    opt_W = SolverFactory('gurobi')
    model_W = ConcreteModel()

    # Sets
    model_W.WT = RangeSet(1, W_WT_max.size) # Regular water treatment units set
    model_W.ST = RangeSet(1, WL_ST_max.size) # Water storage tanks set
    model_W.T = RangeSet(1, W_LOAD.size) # Time intervals set
    model_W.V = RangeSet(1, v_f) # PWL functions set
    model_W.V_minus1 = RangeSet(1, v_f-1) # Set of pump power PWL approximated values

    # Binary Variables
    model_W.u_WW = Var(model_W.T, domain=Binary) # Wastewater plant status indicator
    model_W.u_WT = Var(model_W.WT, model_W.T, domain=Binary) # Regular treatment plant status indicator
    model_W.s_STc = Var(model_W.ST, model_W.T, domain=Binary) # Water storage tank fill pump status indicator
    model_W.s_STd = Var(model_W.ST, model_W.T, domain=Binary) # Water storage tank release valve status indicator
    model_W.alpha_WW = Var(model_W.V_minus1, model_W.T, domain=Binary) # Wastewater Treatment Pump PWL control variable
    model_W.alpha_WT = Var(model_W.V_minus1, model_W.WT, model_W.T, domain=Binary) # WRegular Water Treatment Pump PWL control variable
    model_W.alpha_ST = Var(model_W.V_minus1, model_W.ST, model_W.T, domain=Binary) # Water Storage Pump PWL control variable
    # Continuous Variables
    model_W.W_WW = Var(model_W.T, domain=NonNegativeReals) # Wastewater plant water flow rate output [gal/h]
    model_W.WL_WW = Var(model_W.T, domain=NonNegativeReals) # Wastewater reservoir level [gal]
    model_W.P_WW = Var(model_W.T, domain=NonNegativeReals) # Wastewater plant power consumption [kW]
    model_W.W_WT = Var(model_W.WT, model_W.T, domain=NonNegativeReals) # Regular water treatment plant water flow rate output [gal/h]
    model_W.P_WT = Var(model_W.WT, model_W.T, domain=NonNegativeReals) # Regular water treatment plant power consumption [kW]
    model_W.W_STc = Var(model_W.ST, model_W.T, domain=NonNegativeReals) # Water storage fill flow rate input [gal/h]
    model_W.W_STd = Var(model_W.ST, model_W.T, domain=NonNegativeReals) # Water storage release flow rate output [gal/h]
    model_W.WL_ST = Var(model_W.ST, model_W.T, domain=NonNegativeReals) # Water storage tank level [gal]
    model_W.P_WW_pump = Var(model_W.V_minus1, model_W.T, domain=NonNegativeReals) # Wastewater pump power consumption [kW]
    model_W.P_WT_pump = Var(model_W.V_minus1, model_W.WT, model_W.T, domain=NonNegativeReals) # Regular water treatment pump power consumption [kW]
    model_W.P_ST_pump = Var(model_W.V_minus1, model_W.ST, model_W.T, domain=NonNegativeReals) # Water storage tank pump power consumption [kW]
    model_W.P_water_W_var = Var(model_W.T, domain=NonNegativeReals) # Total power consumption of water-related components [kW]

    # Objective Function
    def obj_fxn_W(model_W): # Eq. (38)
        f = 0
        for t in model_W.T:
            f += lam_SADMM[k,t-1]*(P_water_E_SADMM[k+1,t-1] - model_W.P_water_W_var[t]) + (rho/2)*(P_water_E_SADMM[k+1,t-1] - model_W.P_water_W_var[t])**2 # Lagrangian augmentation
        return f
    model_W.OBJ = Objective(rule=obj_fxn_W, sense=minimize)

    # Constraints
    # Power Consumpion
    def Power_Consumption(model_W, t):
        return model_W.P_water_W_var[t] == model_W.P_WW[t] + model_W.P_WW_pump[v_f-1,t] + sum(model_W.P_WT[w,t] + model_W.P_WT_pump[v_f-1,w,t] for w in model_W.WT) + sum(model_W.P_ST_pump[v_f-1,k,t] for k in model_W.ST)
    model_W.Power_Balance = Constraint(model_W.T, rule=Power_Consumption)
    # Wastewater
    def Wastewater_min_output_rule(model_W, t):
        return W_WW_min*model_W.u_WW[t] <= model_W.W_WW[t]
    model_W.Wastewater_min_output = Constraint(model_W.T, rule=Wastewater_min_output_rule)
    def Wastewater_max_output_rule(model_W, t):
        return model_W.W_WW[t] <= W_WW_max*model_W.u_WW[t]
    model_W.Wastewater_max_output = Constraint(model_W.T, rule=Wastewater_max_output_rule)
    def Wastewater_reservoir_lvl_rule(model_W, t):
        if t == 1:
            return model_W.WL_WW[t] == WL_WW_init + delta_t*(WR_WW[t-1] - model_W.W_WW[t])
        else:
            return model_W.WL_WW[t] == model_W.WL_WW[t-1] + delta_t*(WR_WW[t-1] - model_W.W_WW[t])
    model_W.Wastewater_reservoir_lvl = Constraint(model_W.T, rule=Wastewater_reservoir_lvl_rule)
    def Wastewater_reservoir_limit_rule(model_W, t):
        return model_W.WL_WW[t] <= WL_WW_lim
    model_W.Wastewater_reservoir_limit = Constraint(model_W.T, rule=Wastewater_reservoir_limit_rule)
    def Wastewater_PowerConsumption_rule(model_W, t):
        return model_W.P_WW[t] == C_WW*model_W.W_WW[t]
    model_W.Wastewater_PowerConsumption = Constraint(model_W.T, rule=Wastewater_PowerConsumption_rule)
    def Wastewater_Use_rule(model_W, t):
        return model_W.WL_WW[model_W.T.last()] <= 0.1*WL_WW_lim
    model_W.Wastewater_Use = Constraint(model_W.T, rule=Wastewater_Use_rule) # Restrict end-of-day wastewater level to 10% of its total capacity
    # Regular Treatment
    def RegWater_min_output_rule(model_W, w, t):
        return W_WT_min[w-1]*model_W.u_WT[w,t] <= model_W.W_WT[w,t]
    model_W.RegWater_min_output = Constraint(model_W.WT, model_W.T, rule=RegWater_min_output_rule)
    def RegWater_max_output_rule(model_W, w, t):
        return model_W.W_WT[w,t] <= W_WT_max[w-1]*model_W.u_WT[w,t]
    model_W.RegWater_max_output = Constraint(model_W.WT, model_W.T, rule=RegWater_max_output_rule)
    def RegWater_PowerConsumption_rule(model_W, w, t):
        return model_W.P_WT[w,t] == C_WT[w-1]*model_W.W_WT[w,t]
    model_W.RegWater_PowerConsumption = Constraint(model_W.WT, model_W.T, rule=RegWater_PowerConsumption_rule)
    # Water Storage
    def WaterStorage_min_output_rule(model_W, k ,t):
        return W_ST_min[k-1]*model_W.s_STc[k,t] <= model_W.W_STc[k,t]
    model_W.WaterStorage_min_output = Constraint(model_W.ST, model_W.T, rule=WaterStorage_min_output_rule)
    def WaterStorage_max_output_rule(model_W, k ,t):
        return model_W.W_STc[k,t] <= W_ST_max[k-1]*model_W.s_STc[k,t]
    model_W.WaterStorage_max_output = Constraint(model_W.ST, model_W.T, rule=WaterStorage_max_output_rule)
    def WaterStorage_max_input_rule(model, k ,t):
        return model_W.W_STd[k,t] <= W_ST_max[k-1]*model_W.s_STd[k,t]
    model_W.WaterStorage_max_input = Constraint(model_W.ST, model_W.T, rule=WaterStorage_max_input_rule)
    def WaterStorage_status_rule(model_W, k, t):
        return model_W.s_STc[k,t] + model_W.s_STd[k,t] <= 1
    model_W.WaterStorage_status = Constraint(model_W.ST, model_W.T, rule=WaterStorage_status_rule)
    def WaterStorage_Tank_lvl_rule(model_W, k, t):
        if t == 1:
            return model_W.WL_ST[k,t] == WL_ST_init[k-1] + delta_t*(model_W.W_STc[k,t] - model_W.W_STd[k,t])
        else:
            return model_W.WL_ST[k,t] == model_W.WL_ST[k,t-1] + delta_t*(model_W.W_STc[k,t] - model_W.W_STd[k,t])
    model_W.WaterStorage_Tank_lvl = Constraint(model_W.ST, model_W.T, rule=WaterStorage_Tank_lvl_rule)
    def WaterStorage_min_Tank_lvl_rule(model_W, k, t):
        return WL_ST_min[k-1] <= model_W.WL_ST[k,t]
    model_W.WaterStorage_min_Tank_lvl = Constraint(model_W.ST, model_W.T, rule=WaterStorage_min_Tank_lvl_rule)
    def WaterStorage_max_Tank_lvl_rule(model_W, k, t):
        return model_W.WL_ST[k,t] <= WL_ST_max[k-1]
    model_W.WaterStorage_max_Tank_lvl = Constraint(model_W.ST, model_W.T, rule=WaterStorage_max_Tank_lvl_rule)
    # Water Balance
    def Water_Balance_rule(model_W, t):
        return model_W.W_WW[t] + sum(model_W.W_WT[w,t] for w in model_W.WT) + sum(model_W.W_STd[k,t] - model_W.W_STc[k,t] for k in model_W.ST) == W_LOAD[t-1]
    model_W.Water_Balance = Constraint(model_W.T, rule=Water_Balance_rule)

    # Linearized Pump Constraints for each water resource
    def WastewaterPump_Constraint1_lower_rule(model_W, t):
        return a_WW[0]*model_W.W_WW[t] + b_WW[0]*model_W.u_WW[t] <= model_W.P_WW_pump[1,t]
    model_W.WastewaterPump_Constraint1_lower = Constraint(model_W.T, rule=WastewaterPump_Constraint1_lower_rule)
    def WastewaterPump_Constraint1_upper_rule(model_W, t):
        return model_W.P_WW_pump[1,t] <= a_WW[0]*model_W.W_WW[t] + b_WW[0]*model_W.u_WW[t] + model_W.alpha_WW[1,t]*BigM
    model_W.WastewaterPump_Constraint1_upper = Constraint(model_W.T, rule=WastewaterPump_Constraint1_upper_rule)
    def WastewaterPump_Constraint2_lower_rule(model_W, t):
        return a_WW[1]*model_W.W_WW[t] + b_WW[1]*model_W.u_WW[t] <= model_W.P_WW_pump[1,t]
    model_W.WastewaterPump_Constraint2_lower = Constraint(model_W.T, rule=WastewaterPump_Constraint2_lower_rule)
    def WastewaterPump_Constraint2_upper_rule(model_W, t):
        return model_W.P_WW_pump[1,t] <= a_WW[1]*model_W.W_WW[t] + b_WW[1]*model_W.u_WW[t] + (1 - model_W.alpha_WW[1,t])*BigM
    model_W.WastewaterPump_Constraint2_upper = Constraint(model_W.T, rule=WastewaterPump_Constraint2_upper_rule)
    def WastewaterPump_Constraint3_lower_rule(model_W, t, v):
        if v >= 3:
            return model_W.P_WW_pump[v-2,t] <= model_W.P_WW_pump[v-1,t]
        else:
            return Constraint.Skip
    model_W.WastewaterPump_Constraint3_lower = Constraint(model_W.T, model_W.V, rule=WastewaterPump_Constraint3_lower_rule)
    def WastewaterPump_Constraint3_upper_rule(model_W, t, v):
        if v >= 3:
            return model_W.P_WW_pump[v-1,t] <= model_W.P_WW_pump[v-2,t] + model_W.alpha_WW[v-1,t]*BigM
        else:
            return Constraint.Skip
    model_W.WastewaterPump_Constraint3_upper = Constraint(model_W.T, model_W.V, rule=WastewaterPump_Constraint3_upper_rule)
    def WastewaterPump_Constraint4_lower_rule(model_W, t, v):
        if v >= 3:
            return a_WW[v-1]*model_W.W_WW[t] + b_WW[v-1]*model_W.u_WW[t] <= model_W.P_WW_pump[v-1,t]
        else:
            return Constraint.Skip
    model_W.WastewaterPump_Constraint4_lower = Constraint(model_W.T, model_W.V, rule=WastewaterPump_Constraint4_lower_rule)
    def WastewaterPump_Constraint4_upper_rule(model_W, t, v):
        if v >= 3:
            return model_W.P_WW_pump[v-1,t] <= a_WW[v-1]*model_W.W_WW[t] + b_WW[v-1]*model_W.u_WW[t] + (1 - model_W.alpha_WW[v-1,t])*BigM
        else:
            return Constraint.Skip
    model_W.WastewaterPump_Constraint4_upper = Constraint(model_W.T, model_W.V, rule=WastewaterPump_Constraint4_upper_rule)

    def RegWaterPump_Constraint1_lower_rule(model_W, w, t):
        return a_WT[0,w-1]*model_W.W_WT[w,t] + b_WT[0,w-1]*model_W.u_WT[w,t] <= model_W.P_WT_pump[1,w,t]
    model_W.RegWaterPump_Constraint1_lower = Constraint(model_W.WT, model_W.T, rule=RegWaterPump_Constraint1_lower_rule)
    def RegWaterPump_Constraint1_upper_rule(model_W, w, t):
        return model_W.P_WT_pump[1,w,t] <= a_WT[0,w-1]*model_W.W_WT[w,t] + b_WT[0,w-1]*model_W.u_WT[w,t] + model_W.alpha_WT[1,w,t]*BigM
    model_W.RegWaterPump_Constraint1_upper = Constraint(model_W.WT, model_W.T, rule=RegWaterPump_Constraint1_upper_rule)
    def RegWaterPump_Constraint2_lower_rule(model_W, w, t):
        return a_WT[1,w-1]*model_W.W_WT[w,t] + b_WT[1,w-1]*model_W.u_WT[w,t] <= model_W.P_WT_pump[1,w,t]
    model_W.RegWaterPump_Constraint2_lower = Constraint(model_W.WT, model_W.T, rule=RegWaterPump_Constraint2_lower_rule)
    def RegWaterPump_Constraint2_upper_rule(model_W, w, t):
        return model_W.P_WT_pump[1,w,t] <= a_WT[1,w-1]*model_W.W_WT[w,t] + b_WT[1,w-1]*model_W.u_WT[w,t] + (1 - model_W.alpha_WT[1,w,t])*BigM
    model_W.RegWaterPump_Constraint2_upper = Constraint(model_W.WT, model_W.T, rule=RegWaterPump_Constraint2_upper_rule)
    def RegWaterPump_Constraint3_lower_rule(model_W, w, t, v):
        if v >= 3:
            return model_W.P_WT_pump[v-2,w,t] <= model_W.P_WT_pump[v-1,w,t]
        else:
            return Constraint.Skip
    model_W.RegWaterPump_Constraint3_lower = Constraint(model_W.WT, model_W.T, model_W.V, rule=RegWaterPump_Constraint3_lower_rule)
    def RegWaterPump_Constraint3_upper_rule(model_W, w, t, v):
        if v >= 3:
            return model_W.P_WT_pump[v-1,w,t] <= model_W.P_WT_pump[v-2,w,t] + model_W.alpha_WT[v-1,w,t]*BigM
        else:
            return Constraint.Skip
    model_W.RegWaterPump_Constraint3_upper = Constraint(model_W.WT, model_W.T, model_W.V, rule=RegWaterPump_Constraint3_upper_rule)
    def RegWaterPump_Constraint4_lower_rule(model_W, w, t, v):
        if v >= 3:
            return a_WT[v-1,w-1]*model_W.W_WT[w,t] + b_WT[v-1,w-1]*model_W.u_WT[w,t] <= model_W.P_WT_pump[v-1,w,t]
        else:
            return Constraint.Skip
    model_W.RegWaterPump_Constraint4_lower = Constraint(model_W.WT, model_W.T, model_W.V, rule=RegWaterPump_Constraint4_lower_rule)
    def RegWaterPump_Constraint4_upper_rule(model_W, w, t, v):
        if v >= 3:
            return model_W.P_WT_pump[v-1,w,t] <= a_WT[v-1,w-1]*model_W.W_WT[w,t] + b_WT[v-1,w-1]*model_W.u_WT[w,t] + (1 - model_W.alpha_WT[v-1,w,t])*BigM
        else:
            return Constraint.Skip
    model_W.RegWaterPump_Constraint4_upper = Constraint(model_W.WT, model_W.T, model_W.V, rule=RegWaterPump_Constraint4_upper_rule)

    def WaterStoragePump_Constraint1_lower_rule(model_W, k, t):
        return a_ST[0,k-1]*model_W.W_STc[k,t] + b_ST[0,k-1]*model_W.s_STc[k,t] <= model_W.P_ST_pump[1,k,t]
    model_W.WaterStoragePump_Constraint1_lower = Constraint(model_W.ST, model_W.T, rule=WaterStoragePump_Constraint1_lower_rule)
    def WaterStoragePump_Constraint1_upper_rule(model_W, k, t):
        return model_W.P_ST_pump[1,k,t] <= a_ST[0,k-1]*model_W.W_STc[k,t] + b_ST[0,k-1]*model_W.s_STc[k,t] + model_W.alpha_ST[1,k,t]*BigM
    model_W.WaterStoragePump_Constraint1_upper = Constraint(model_W.ST, model_W.T, rule=WaterStoragePump_Constraint1_upper_rule)
    def WaterStoragePump_Constraint2_lower_rule(model_W, k, t):
        return a_ST[1,k-1]*model_W.W_STc[k,t] + b_ST[1,k-1] <= model_W.P_ST_pump[1,k,t]
    model_W.WaterStoragePump_Constraint2_lower = Constraint(model_W.ST, model_W.T, rule=WaterStoragePump_Constraint2_lower_rule)
    def WaterStoragePump_Constraint2_upper_rule(model_W, k, t):
        return model_W.P_ST_pump[1,k,t] <= a_WT[1,k-1]*model_W.W_STc[k,t] + b_ST[1,k-1]*model_W.s_STc[k,t] + (1 - model_W.alpha_ST[1,k,t])*BigM
    model_W.WaterStoragePump_Constraint2_upper = Constraint(model_W.ST, model_W.T, rule=WaterStoragePump_Constraint2_upper_rule)
    def WaterStoragePump_Constraint3_lower_rule(model_W, k, t, v):
        if v >= 3:
            return model_W.P_ST_pump[v-2,k,t] <= model_W.P_WT_pump[v-1,k,t]
        else:
            return Constraint.Skip
    model_W.WaterStoragePump_Constraint3_lower = Constraint(model_W.ST, model_W.T, model_W.V, rule=WaterStoragePump_Constraint3_lower_rule)
    def WaterStoragePump_Constraint3_upper_rule(model_W, k, t, v):
        if v >= 3:
            return model_W.P_ST_pump[v-1,k,t] <= model_W.P_ST_pump[v-2,k,t] + model_W.alpha_ST[v-1,k,t]*BigM
        else:
            return Constraint.Skip
    model_W.WaterStoragePump_Constraint3_upper = Constraint(model_W.ST, model_W.T, model_W.V, rule=WaterStoragePump_Constraint3_upper_rule)
    def WaterStoragePump_Constraint4_lower_rule(model_W, k, t, v):
        if v >= 3:
            return a_ST[v-1,k-1]*model_W.W_STc[k,t] + b_ST[v-1,k-1]*model_W.s_STc[k,t] <= model_W.P_ST_pump[v-1,k,t]
        else:
            return Constraint.Skip
    model_W.WaterStoragePump_Constraint4_lower = Constraint(model_W.ST, model_W.T, model_W.V, rule=RegWaterPump_Constraint4_lower_rule)
    def WaterStoragePump_Constraint4_upper_rule(model_W, k, t, v):
        if v >= 3:
            return model_W.P_ST_pump[v-1,k,t] <= a_ST[v-1,k-1]*model_W.W_STc[k,t] + b_ST[v-1,k-1]*model_W.s_STc[k,t] + (1 - model_W.alpha_ST[v-1,k,t])*BigM
        else:
            return Constraint.Skip
    model_W.WaterStoragePump_Constraint4_upper = Constraint(model_W.ST, model_W.T, model_W.V, rule=WaterStoragePump_Constraint4_upper_rule)
    
    # MWM Optimization Solution
    Results_W = opt_W.solve(model_W)
    OBJ_W_SADMM[k+1] = value(model_W.OBJ)
    for t in range(W_LOAD.size):
        P_water_W_SADMM[k+1,t] = value(model_W.P_water_W_var[t+1])
        
    
    # Global Objective Value
    for t in range(P_LOAD.size):
        OBJ_global_SADMM[k+1] = OBJ_E_SADMM[k+1] + OBJ_W_SADMM[k+1]
        
    # Lagrange Multiplier and Residuals Update
    for t in range(P_LOAD.size):
        lam_SADMM[k+1,t] = lam_SADMM[k,t] + rho*(P_water_E_SADMM[k+1,t] - P_water_W_SADMM[k+1,t]) # Updating Lagrange multiplier
        r_SADMM[k+1,t] = P_water_E_SADMM[k+1,t] - P_water_W_SADMM[k+1,t] # Updating primal residual
        s_SADMM[k+1,t] = (P_water_E_SADMM[k+1,t] - P_water_W_SADMM[k+1,t]) - (P_water_E_SADMM[k,t] - P_water_W_SADMM[k,t]) # Updating dual residual
    eps_SADMM[k+1] = sqrt(np.sum(r_SADMM[k+1,:])**2 + np.sum(s_SADMM[k+1,:])**2) # Calculating solution feasibility metric
    
    k += 1 # Next iteration
    
    # Objective-Based Convergence Check
    if eps_SADMM[k] < eps_threshold: # Solution feasibility converged under threshold ?
            Check = 0
            
    if k >= 10000:
        break
    
    # Allocating space for next iteration
    if Check > 0:
        P_water_E_SADMM = np.vstack((P_water_E_SADMM, np.zeros(P_LOAD.size)))
        P_water_W_SADMM = np.vstack((P_water_W_SADMM, np.zeros(W_LOAD.size)))
        lam_SADMM = np.vstack((lam_SADMM, np.zeros(P_LOAD.size)))
        r_SADMM = np.vstack((r_SADMM, np.ones(P_LOAD.size)))
        s_SADMM = np.vstack((s_SADMM, np.ones(P_LOAD.size)))
        eps_SADMM = np.append(eps_SADMM,1)
        OBJ_E_SADMM = np.append(OBJ_E_SADMM,0)
        OBJ_W_SADMM = np.append(OBJ_W_SADMM,0)
        OBJ_global_SADMM = np.append(OBJ_global_SADMM,0)
    
# Main While Loop Ends

D_end_time = time.time()
D_COMP_TIME = D_end_time - D_start_time
if Check == 0:
    print(f"Standard ADMM Algorithm finished in {k} iterations.")
else:
    print(f"Standard ADMM Algorithm did no finish within {k} iterations.")
print("\nComputation time: {:.5f} sec.".format(D_COMP_TIME))
    

In [ ]:
print("Objective Value: {:.7f}".format(OBJ_global_SADMM[-1]))
print("Normalized Obj. Value: {:.7f}".format(OBJ_global_SADMM[-1]/value(model_CMWEN.OBJ)))
print("Final Solution Feasibility: {:.6e}".format(eps_SADMM[-1]))

In [ ]:
# Total MWM Energy Consumption
print('Energy Consumption: {:0.3f} kWh'.format(sum(P_water_W_SADMM[-1,t-1] for t in model_CMWEN.T)))

In [ ]:
# Total WMW Operation Power Consumption
plt.figure(figsize=(5, 4))
X = np.zeros(W_LOAD.size)
Y = np.zeros(W_LOAD.size)
for t in range(0, W_LOAD.size):
    X[t-1] = t+1
    Y[t-1] = value(P_water_W_SADMM[-1,t])

plt.bar(X,Y,color='orangered')
plt.xticks(np.arange(2, max(X)+1, 2))
plt.xlabel('Hour')
plt.ylabel('Power [kWh]')
plt.title('Water Management Power Consumption')
plt.grid(True)
plt.gca().set_axisbelow(True)

In [ ]:
# Solution Feasibility Convergence
start = 4 # State initial iteration to look at
plt.figure(figsize=(5, 4))
X = np.array(range(start,k+1))
plt.plot(X,eps_SADMM[start:k+1],color='darkblue')
plt.xlabel('Iteration (k)')
plt.ylabel('Solution Feasibility (Ɛ)')
plt.grid(True)
plt.gca().set_axisbelow(True)

In [ ]:
# Normalized Objective Value Convergence
start = 4 # State initial iteration to look at
plt.figure(figsize=(5, 4))
X = np.array(range(start,k+1))
plt.plot(X,OBJ_global_SADMM[start:k+1]/value(model_CMWEN.OBJ),color='darkred')
plt.xlabel('Iteration (k)')
plt.ylabel('Normalized Obj. Val.')
plt.grid(True)
plt.gca().set_axisbelow(True)

### Objective-Based ADMM

In [ ]:
# DECENTRALIZED OPTIMIZATION: OB-ADMM ALGORITHM
D_start_time = time.time()

# Initializing algorithm variables
P_water_E_OBADMM = np.zeros([2, P_LOAD.size]) # Water management power consumption duplicated global variales
P_water_W_OBADMM = np.zeros([2, W_LOAD.size])
lam_OBADMM = np.zeros([2, P_LOAD.size]) # Lagrange multiplier
r_OBADMM = np.ones([2, P_LOAD.size]) # Primal residual
s_OBADMM = np.ones([2, P_LOAD.size]) # Dual residual
eps_OBADMM = np.ones(2) # Solution feasibility metric
# OB-ADMM Hyperparameters
rho = 0.01 # penalty
k_s = 50 # Iteration offset
beta = 0.0001 # Objective value rate of change preset threshold
# Individual Objective Value Arrays
OBJ_E_OBADMM = np.zeros(2)
OBJ_W_OBADMM = np.zeros(2)
OBJ_global_OBADMM = np.zeros(2) # Global objective value

k = 0
# OB-ADMM iteration loop
Check = 1
while Check:
    
    # MEM LOCAL OPTIMIZATION MODEL
    opt_E = SolverFactory('gurobi')
    model_E = ConcreteModel()

    # Sets
    model_E.G = RangeSet(1, C_G.size) # Generators set
    model_E.ES = RangeSet(1, P_ES_lim.size) # Energy storage units set
    model_E.T = RangeSet(1, P_LOAD.size) # Time intervals set

    # Binary Variables
    model_E.u_G = Var(model_E.G, model_E.T, domain=Binary) # Generator status indicatior
    model_E.e_ESc = Var(model_E.ES, model_E.T, domain=Binary) # Energy storage charging status indicator
    model_E.e_ESd = Var(model_E.ES, model_E.T, domain=Binary) # Energy storage discharging status indicator
    model_E.p_plus = Var(model_E.T, domain=Binary) # Main grid power import indicator
    model_E.p_minus = Var(model_E.T, domain=Binary) # Main grid power export indicator
    # Continuous Variables
    model_E.P_G = Var(model_E.G, model_E.T, domain=NonNegativeReals) # Generator power output [kW] 
    model_E.P_ESc = Var(model_E.ES, model_E.T, domain=NonNegativeReals) # Energy storage charging input [kW]
    model_E.P_ESd = Var(model_E.ES, model_E.T, domain=NonNegativeReals) # Energy storage discharging output [kW]
    model_E.EL_ES = Var(model_E.ES, model_E.T, domain=NonNegativeReals) # Energy storage charge level [kWh]
    model_E.P_grid_plus = Var(model_E.T, domain=NonNegativeReals) # Main grid power import [kW]
    model_E.P_grid_minus = Var(model_E.T, domain=NonNegativeReals) # Main grid power export [kW]
    model_E.P_water_E_var = Var(model_E.T, domain=NonNegativeReals) # Power consumption of water components [kW]

    # Objective Function
    def obj_fxn_E(model_E): # Eq. (37)
        f = 0
        for t in model_E.T:
            for g in model_E.G:
                f += delta_t*(NL_G[g-1]*model_E.u_G[g,t] + C_G[g-1]*model_E.P_G[g,t])
            f += C_grid_plus[t-1]*model_E.P_grid_plus[t] - C_grid_minus[t-1]*model_E.P_grid_minus[t]
            f += lam_OBADMM[k,t-1]*(model_E.P_water_E_var[t] - P_water_W_OBADMM[k,t-1]) + (rho/2)*(model_E.P_water_E_var[t] - P_water_W_OBADMM[k,t-1])**2 # Lagrangian augmentation
        return f
    model_E.OBJ = Objective(rule=obj_fxn_E, sense=minimize)

    # Constraints
    # Generators
    def Gen_min_output_rule(model_E, g, t):
        return P_G_min[g-1]*model_E.u_G[g,t] <= model_E.P_G[g,t]
    model_E.Gen_min_output = Constraint(model_E.G, model_E.T, rule=Gen_min_output_rule)
    def Gen_max_output_rule(model_E, g, t):
        return model_E.P_G[g,t] <= P_G_max[g-1]*model_E.u_G[g,t]
    model_E.Gen_max_output = Constraint(model_E.G, model_E.T, rule=Gen_max_output_rule)
    # Energy Storage
    def ES_charge_limit_rule(model_E, b, t):
        return model_E.P_ESc[b,t] <= P_ES_lim[b-1]*model_E.e_ESc[b,t]
    model_E.ES_charge_limit = Constraint(model_E.ES, model_E.T, rule=ES_charge_limit_rule)
    def ES_discharge_limit_rule(model_E, b, t):
        return model_E.P_ESd[b,t] <= P_ES_lim[b-1]*model_E.e_ESd[b,t]
    model_E.ES_discharge_limit = Constraint(model_E.ES, model_E.T, rule=ES_discharge_limit_rule)
    def ES_status_rule(model_E, b, t):
        return model_E.e_ESc[b,t] + model_E.e_ESd[b,t] <= 1
    model_E.ES_status = Constraint(model_E.ES, model_E.T, rule=ES_status_rule)
    def ES_charge_lvl_rule(model_E, b, t):
        if t == 1:
            return model_E.EL_ES[b,t] == EL_ES_init[b-1] + delta_t*(eff_ESc[b-1]*model_E.P_ESc[b,t] - model_E.P_ESd[b,t]/eff_ESd[b-1])
        else:
            return model_E.EL_ES[b,t] == model_E.EL_ES[b,t-1] + delta_t*(eff_ESc[b-1]*model_E.P_ESc[b,t] - model_E.P_ESd[b,t]/eff_ESd[b-1])
    model_E.ES_charge_lvl = Constraint(model_E.ES, model_E.T, rule=ES_charge_lvl_rule)
    def ES_min_charge_lvl_rule(model_E, b , t):
        return EL_ES_min[b-1] <= model_E.EL_ES[b,t]
    model_E.ES_min_charge_lvl = Constraint(model_E.ES, model_E.T, rule=ES_min_charge_lvl_rule)
    def ES_max_charge_lvl_rule(model_E, b , t):
        return model_E.EL_ES[b,t] <= EL_ES_max[b-1]
    model_E.ES_max_charge_lvl = Constraint(model_E.ES, model_E.T, rule=ES_max_charge_lvl_rule)
    # Main Grid
    def Grid_import_limit_rule(model_E, t):
        return model_E.P_grid_plus[t] <= P_grid_lim*model_E.p_plus[t]
    model_E.Grid_import_limit = Constraint(model_E.T, rule=Grid_import_limit_rule)
    def Grid_export_limit_rule(model_E, t):
        return model_E.P_grid_minus[t] <= P_grid_lim*model_E.p_minus[t]
    model_E.Grid_export_limit = Constraint(model_E.T, rule=Grid_export_limit_rule)
    def Grid_status_rule(model_E, t):
        return model_E.p_plus[t] + model_E.p_minus[t] <= 1
    model_E.Grid_status = Constraint(model_E.T, rule=Grid_status_rule)
    # Power Balance
    def Power_Balance_rule(model_E, t):
        return sum(model_E.P_G[g,t] for g in model_E.G) + sum(model_E.P_ESd[b,t] - model_E.P_ESc[b,t] for b in model_E.ES) + model_E.P_grid_plus[t] - model_E.P_grid_minus[t] == P_LOAD[t-1] - P_SOLAR[t-1] + model_E.P_water_E_var[t]
    model_E.Power_Balance = Constraint(model_E.T, rule=Power_Balance_rule)
    
    # MEM Optimization Solution
    Results_E = opt_E.solve(model_E)
    OBJ_E_OBADMM[k+1] = value(model_E.OBJ)
    for t in range(P_LOAD.size):
        P_water_E_OBADMM[k+1,t] = value(model_E.P_water_E_var[t+1])
        
        
    # MWM LOCAL OPTIMIZATION MODEL
    opt_W = SolverFactory('gurobi')
    model_W = ConcreteModel()

    # Sets
    model_W.WT = RangeSet(1, W_WT_max.size) # Regular water treatment units set
    model_W.ST = RangeSet(1, WL_ST_max.size) # Water storage tanks set
    model_W.T = RangeSet(1, W_LOAD.size) # Time intervals set
    model_W.V = RangeSet(1, v_f) # PWL functions set
    model_W.V_minus1 = RangeSet(1, v_f-1) # Set of pump power PWL approximated values

    # Binary Variables
    model_W.u_WW = Var(model_W.T, domain=Binary) # Wastewater plant status indicator
    model_W.u_WT = Var(model_W.WT, model_W.T, domain=Binary) # Regular treatment plant status indicator
    model_W.s_STc = Var(model_W.ST, model_W.T, domain=Binary) # Water storage tank fill pump status indicator
    model_W.s_STd = Var(model_W.ST, model_W.T, domain=Binary) # Water storage tank release valve status indicator
    model_W.alpha_WW = Var(model_W.V_minus1, model_W.T, domain=Binary) # Wastewater Treatment Pump PWL control variable
    model_W.alpha_WT = Var(model_W.V_minus1, model_W.WT, model_W.T, domain=Binary) # WRegular Water Treatment Pump PWL control variable
    model_W.alpha_ST = Var(model_W.V_minus1, model_W.ST, model_W.T, domain=Binary) # Water Storage Pump PWL control variable
    # Continuous Variables
    model_W.W_WW = Var(model_W.T, domain=NonNegativeReals) # Wastewater plant water flow rate output [gal/h]
    model_W.WL_WW = Var(model_W.T, domain=NonNegativeReals) # Wastewater reservoir level [gal]
    model_W.P_WW = Var(model_W.T, domain=NonNegativeReals) # Wastewater plant power consumption [kW]
    model_W.W_WT = Var(model_W.WT, model_W.T, domain=NonNegativeReals) # Regular water treatment plant water flow rate output [gal/h]
    model_W.P_WT = Var(model_W.WT, model_W.T, domain=NonNegativeReals) # Regular water treatment plant power consumption [kW]
    model_W.W_STc = Var(model_W.ST, model_W.T, domain=NonNegativeReals) # Water storage fill flow rate input [gal/h]
    model_W.W_STd = Var(model_W.ST, model_W.T, domain=NonNegativeReals) # Water storage release flow rate output [gal/h]
    model_W.WL_ST = Var(model_W.ST, model_W.T, domain=NonNegativeReals) # Water storage tank level [gal]
    model_W.P_WW_pump = Var(model_W.V_minus1, model_W.T, domain=NonNegativeReals) # Wastewater pump power consumption [kW]
    model_W.P_WT_pump = Var(model_W.V_minus1, model_W.WT, model_W.T, domain=NonNegativeReals) # Regular water treatment pump power consumption [kW]
    model_W.P_ST_pump = Var(model_W.V_minus1, model_W.ST, model_W.T, domain=NonNegativeReals) # Water storage tank pump power consumption [kW]
    model_W.P_water_W_var = Var(model_W.T, domain=NonNegativeReals) # Total power consumption of water-related components [kW]

    # Objective Function
    def obj_fxn_W(model_W): # Eq. (38)
        f = 0
        for t in model_W.T:
            f += lam_OBADMM[k,t-1]*(P_water_E_OBADMM[k+1,t-1] - model_W.P_water_W_var[t]) + (rho/2)*(P_water_E_OBADMM[k+1,t-1] - model_W.P_water_W_var[t])**2 # Lagrangian augmentation
        return f
    model_W.OBJ = Objective(rule=obj_fxn_W, sense=minimize)

    # Constraints
    # Power Consumption
    def Power_Consumption(model_W, t):
        return model_W.P_water_W_var[t] == model_W.P_WW[t] + model_W.P_WW_pump[v_f-1,t] + sum(model_W.P_WT[w,t] + model_W.P_WT_pump[v_f-1,w,t] for w in model_W.WT) + sum(model_W.P_ST_pump[v_f-1,k,t] for k in model_W.ST)
    model_W.Power_Balance = Constraint(model_W.T, rule=Power_Consumption)
    # Wastewater
    def Wastewater_min_output_rule(model_W, t):
        return W_WW_min*model_W.u_WW[t] <= model_W.W_WW[t]
    model_W.Wastewater_min_output = Constraint(model_W.T, rule=Wastewater_min_output_rule)
    def Wastewater_max_output_rule(model_W, t):
        return model_W.W_WW[t] <= W_WW_max*model_W.u_WW[t]
    model_W.Wastewater_max_output = Constraint(model_W.T, rule=Wastewater_max_output_rule)
    def Wastewater_reservoir_lvl_rule(model_W, t):
        if t == 1:
            return model_W.WL_WW[t] == WL_WW_init + delta_t*(WR_WW[t-1] - model_W.W_WW[t])
        else:
            return model_W.WL_WW[t] == model_W.WL_WW[t-1] + delta_t*(WR_WW[t-1] - model_W.W_WW[t])
    model_W.Wastewater_reservoir_lvl = Constraint(model_W.T, rule=Wastewater_reservoir_lvl_rule)
    def Wastewater_reservoir_limit_rule(model_W, t):
        return model_W.WL_WW[t] <= WL_WW_lim
    model_W.Wastewater_reservoir_limit = Constraint(model_W.T, rule=Wastewater_reservoir_limit_rule)
    def Wastewater_PowerConsumption_rule(model_W, t):
        return model_W.P_WW[t] == C_WW*model_W.W_WW[t]
    model_W.Wastewater_PowerConsumption = Constraint(model_W.T, rule=Wastewater_PowerConsumption_rule)
    def Wastewater_Use_rule(model_W, t):
        return model_W.WL_WW[model_W.T.last()] <= 0.1*WL_WW_lim
    model_W.Wastewater_Use = Constraint(model_W.T, rule=Wastewater_Use_rule) # Restrict end-of-day wastewater level to 10% of its total capacity
    # Regular Treatment
    def RegWater_min_output_rule(model_W, w, t):
        return W_WT_min[w-1]*model_W.u_WT[w,t] <= model_W.W_WT[w,t]
    model_W.RegWater_min_output = Constraint(model_W.WT, model_W.T, rule=RegWater_min_output_rule)
    def RegWater_max_output_rule(model_W, w, t):
        return model_W.W_WT[w,t] <= W_WT_max[w-1]*model_W.u_WT[w,t]
    model_W.RegWater_max_output = Constraint(model_W.WT, model_W.T, rule=RegWater_max_output_rule)
    def RegWater_PowerConsumption_rule(model_W, w, t):
        return model_W.P_WT[w,t] == C_WT[w-1]*model_W.W_WT[w,t]
    model_W.RegWater_PowerConsumption = Constraint(model_W.WT, model_W.T, rule=RegWater_PowerConsumption_rule)
    # Water Storage
    def WaterStorage_min_output_rule(model_W, k ,t):
        return W_ST_min[k-1]*model_W.s_STc[k,t] <= model_W.W_STc[k,t]
    model_W.WaterStorage_min_output = Constraint(model_W.ST, model_W.T, rule=WaterStorage_min_output_rule)
    def WaterStorage_max_output_rule(model_W, k ,t):
        return model_W.W_STc[k,t] <= W_ST_max[k-1]*model_W.s_STc[k,t]
    model_W.WaterStorage_max_output = Constraint(model_W.ST, model_W.T, rule=WaterStorage_max_output_rule)
    def WaterStorage_max_input_rule(model, k ,t):
        return model_W.W_STd[k,t] <= W_ST_max[k-1]*model_W.s_STd[k,t]
    model_W.WaterStorage_max_input = Constraint(model_W.ST, model_W.T, rule=WaterStorage_max_input_rule)
    def WaterStorage_status_rule(model_W, k, t):
        return model_W.s_STc[k,t] + model_W.s_STd[k,t] <= 1
    model_W.WaterStorage_status = Constraint(model_W.ST, model_W.T, rule=WaterStorage_status_rule)
    def WaterStorage_Tank_lvl_rule(model_W, k, t):
        if t == 1:
            return model_W.WL_ST[k,t] == WL_ST_init[k-1] + delta_t*(model_W.W_STc[k,t] - model_W.W_STd[k,t])
        else:
            return model_W.WL_ST[k,t] == model_W.WL_ST[k,t-1] + delta_t*(model_W.W_STc[k,t] - model_W.W_STd[k,t])
    model_W.WaterStorage_Tank_lvl = Constraint(model_W.ST, model_W.T, rule=WaterStorage_Tank_lvl_rule)
    def WaterStorage_min_Tank_lvl_rule(model_W, k, t):
        return WL_ST_min[k-1] <= model_W.WL_ST[k,t]
    model_W.WaterStorage_min_Tank_lvl = Constraint(model_W.ST, model_W.T, rule=WaterStorage_min_Tank_lvl_rule)
    def WaterStorage_max_Tank_lvl_rule(model_W, k, t):
        return model_W.WL_ST[k,t] <= WL_ST_max[k-1]
    model_W.WaterStorage_max_Tank_lvl = Constraint(model_W.ST, model_W.T, rule=WaterStorage_max_Tank_lvl_rule)
    # Water Balance
    def Water_Balance_rule(model_W, t):
        return model_W.W_WW[t] + sum(model_W.W_WT[w,t] for w in model_W.WT) + sum(model_W.W_STd[k,t] - model_W.W_STc[k,t] for k in model_W.ST) == W_LOAD[t-1]
    model_W.Water_Balance = Constraint(model_W.T, rule=Water_Balance_rule)

    # Linearized Pump Constraints for each water resource
    def WastewaterPump_Constraint1_lower_rule(model_W, t):
        return a_WW[0]*model_W.W_WW[t] + b_WW[0]*model_W.u_WW[t] <= model_W.P_WW_pump[1,t]
    model_W.WastewaterPump_Constraint1_lower = Constraint(model_W.T, rule=WastewaterPump_Constraint1_lower_rule)
    def WastewaterPump_Constraint1_upper_rule(model_W, t):
        return model_W.P_WW_pump[1,t] <= a_WW[0]*model_W.W_WW[t] + b_WW[0]*model_W.u_WW[t] + model_W.alpha_WW[1,t]*BigM
    model_W.WastewaterPump_Constraint1_upper = Constraint(model_W.T, rule=WastewaterPump_Constraint1_upper_rule)
    def WastewaterPump_Constraint2_lower_rule(model_W, t):
        return a_WW[1]*model_W.W_WW[t] + b_WW[1]*model_W.u_WW[t] <= model_W.P_WW_pump[1,t]
    model_W.WastewaterPump_Constraint2_lower = Constraint(model_W.T, rule=WastewaterPump_Constraint2_lower_rule)
    def WastewaterPump_Constraint2_upper_rule(model_W, t):
        return model_W.P_WW_pump[1,t] <= a_WW[1]*model_W.W_WW[t] + b_WW[1]*model_W.u_WW[t] + (1 - model_W.alpha_WW[1,t])*BigM
    model_W.WastewaterPump_Constraint2_upper = Constraint(model_W.T, rule=WastewaterPump_Constraint2_upper_rule)
    def WastewaterPump_Constraint3_lower_rule(model_W, t, v):
        if v >= 3:
            return model_W.P_WW_pump[v-2,t] <= model_W.P_WW_pump[v-1,t]
        else:
            return Constraint.Skip
    model_W.WastewaterPump_Constraint3_lower = Constraint(model_W.T, model_W.V, rule=WastewaterPump_Constraint3_lower_rule)
    def WastewaterPump_Constraint3_upper_rule(model_W, t, v):
        if v >= 3:
            return model_W.P_WW_pump[v-1,t] <= model_W.P_WW_pump[v-2,t] + model_W.alpha_WW[v-1,t]*BigM
        else:
            return Constraint.Skip
    model_W.WastewaterPump_Constraint3_upper = Constraint(model_W.T, model_W.V, rule=WastewaterPump_Constraint3_upper_rule)
    def WastewaterPump_Constraint4_lower_rule(model_W, t, v):
        if v >= 3:
            return a_WW[v-1]*model_W.W_WW[t] + b_WW[v-1]*model_W.u_WW[t] <= model_W.P_WW_pump[v-1,t]
        else:
            return Constraint.Skip
    model_W.WastewaterPump_Constraint4_lower = Constraint(model_W.T, model_W.V, rule=WastewaterPump_Constraint4_lower_rule)
    def WastewaterPump_Constraint4_upper_rule(model_W, t, v):
        if v >= 3:
            return model_W.P_WW_pump[v-1,t] <= a_WW[v-1]*model_W.W_WW[t] + b_WW[v-1]*model_W.u_WW[t] + (1 - model_W.alpha_WW[v-1,t])*BigM
        else:
            return Constraint.Skip
    model_W.WastewaterPump_Constraint4_upper = Constraint(model_W.T, model_W.V, rule=WastewaterPump_Constraint4_upper_rule)

    def RegWaterPump_Constraint1_lower_rule(model_W, w, t):
        return a_WT[0,w-1]*model_W.W_WT[w,t] + b_WT[0,w-1]*model_W.u_WT[w,t] <= model_W.P_WT_pump[1,w,t]
    model_W.RegWaterPump_Constraint1_lower = Constraint(model_W.WT, model_W.T, rule=RegWaterPump_Constraint1_lower_rule)
    def RegWaterPump_Constraint1_upper_rule(model_W, w, t):
        return model_W.P_WT_pump[1,w,t] <= a_WT[0,w-1]*model_W.W_WT[w,t] + b_WT[0,w-1]*model_W.u_WT[w,t] + model_W.alpha_WT[1,w,t]*BigM
    model_W.RegWaterPump_Constraint1_upper = Constraint(model_W.WT, model_W.T, rule=RegWaterPump_Constraint1_upper_rule)
    def RegWaterPump_Constraint2_lower_rule(model_W, w, t):
        return a_WT[1,w-1]*model_W.W_WT[w,t] + b_WT[1,w-1]*model_W.u_WT[w,t] <= model_W.P_WT_pump[1,w,t]
    model_W.RegWaterPump_Constraint2_lower = Constraint(model_W.WT, model_W.T, rule=RegWaterPump_Constraint2_lower_rule)
    def RegWaterPump_Constraint2_upper_rule(model_W, w, t):
        return model_W.P_WT_pump[1,w,t] <= a_WT[1,w-1]*model_W.W_WT[w,t] + b_WT[1,w-1]*model_W.u_WT[w,t] + (1 - model_W.alpha_WT[1,w,t])*BigM
    model_W.RegWaterPump_Constraint2_upper = Constraint(model_W.WT, model_W.T, rule=RegWaterPump_Constraint2_upper_rule)
    def RegWaterPump_Constraint3_lower_rule(model_W, w, t, v):
        if v >= 3:
            return model_W.P_WT_pump[v-2,w,t] <= model_W.P_WT_pump[v-1,w,t]
        else:
            return Constraint.Skip
    model_W.RegWaterPump_Constraint3_lower = Constraint(model_W.WT, model_W.T, model_W.V, rule=RegWaterPump_Constraint3_lower_rule)
    def RegWaterPump_Constraint3_upper_rule(model_W, w, t, v):
        if v >= 3:
            return model_W.P_WT_pump[v-1,w,t] <= model_W.P_WT_pump[v-2,w,t] + model_W.alpha_WT[v-1,w,t]*BigM
        else:
            return Constraint.Skip
    model_W.RegWaterPump_Constraint3_upper = Constraint(model_W.WT, model_W.T, model_W.V, rule=RegWaterPump_Constraint3_upper_rule)
    def RegWaterPump_Constraint4_lower_rule(model_W, w, t, v):
        if v >= 3:
            return a_WT[v-1,w-1]*model_W.W_WT[w,t] + b_WT[v-1,w-1]*model_W.u_WT[w,t] <= model_W.P_WT_pump[v-1,w,t]
        else:
            return Constraint.Skip
    model_W.RegWaterPump_Constraint4_lower = Constraint(model_W.WT, model_W.T, model_W.V, rule=RegWaterPump_Constraint4_lower_rule)
    def RegWaterPump_Constraint4_upper_rule(model_W, w, t, v):
        if v >= 3:
            return model_W.P_WT_pump[v-1,w,t] <= a_WT[v-1,w-1]*model_W.W_WT[w,t] + b_WT[v-1,w-1]*model_W.u_WT[w,t] + (1 - model_W.alpha_WT[v-1,w,t])*BigM
        else:
            return Constraint.Skip
    model_W.RegWaterPump_Constraint4_upper = Constraint(model_W.WT, model_W.T, model_W.V, rule=RegWaterPump_Constraint4_upper_rule)

    def WaterStoragePump_Constraint1_lower_rule(model_W, k, t):
        return a_ST[0,k-1]*model_W.W_STc[k,t] + b_ST[0,k-1]*model_W.s_STc[k,t] <= model_W.P_ST_pump[1,k,t]
    model_W.WaterStoragePump_Constraint1_lower = Constraint(model_W.ST, model_W.T, rule=WaterStoragePump_Constraint1_lower_rule)
    def WaterStoragePump_Constraint1_upper_rule(model_W, k, t):
        return model_W.P_ST_pump[1,k,t] <= a_ST[0,k-1]*model_W.W_STc[k,t] + b_ST[0,k-1]*model_W.s_STc[k,t] + model_W.alpha_ST[1,k,t]*BigM
    model_W.WaterStoragePump_Constraint1_upper = Constraint(model_W.ST, model_W.T, rule=WaterStoragePump_Constraint1_upper_rule)
    def WaterStoragePump_Constraint2_lower_rule(model_W, k, t):
        return a_ST[1,k-1]*model_W.W_STc[k,t] + b_ST[1,k-1] <= model_W.P_ST_pump[1,k,t]
    model_W.WaterStoragePump_Constraint2_lower = Constraint(model_W.ST, model_W.T, rule=WaterStoragePump_Constraint2_lower_rule)
    def WaterStoragePump_Constraint2_upper_rule(model_W, k, t):
        return model_W.P_ST_pump[1,k,t] <= a_WT[1,k-1]*model_W.W_STc[k,t] + b_ST[1,k-1]*model_W.s_STc[k,t] + (1 - model_W.alpha_ST[1,k,t])*BigM
    model_W.WaterStoragePump_Constraint2_upper = Constraint(model_W.ST, model_W.T, rule=WaterStoragePump_Constraint2_upper_rule)
    def WaterStoragePump_Constraint3_lower_rule(model_W, k, t, v):
        if v >= 3:
            return model_W.P_ST_pump[v-2,k,t] <= model_W.P_WT_pump[v-1,k,t]
        else:
            return Constraint.Skip
    model_W.WaterStoragePump_Constraint3_lower = Constraint(model_W.ST, model_W.T, model_W.V, rule=WaterStoragePump_Constraint3_lower_rule)
    def WaterStoragePump_Constraint3_upper_rule(model_W, k, t, v):
        if v >= 3:
            return model_W.P_ST_pump[v-1,k,t] <= model_W.P_ST_pump[v-2,k,t] + model_W.alpha_ST[v-1,k,t]*BigM
        else:
            return Constraint.Skip
    model_W.WaterStoragePump_Constraint3_upper = Constraint(model_W.ST, model_W.T, model_W.V, rule=WaterStoragePump_Constraint3_upper_rule)
    def WaterStoragePump_Constraint4_lower_rule(model_W, k, t, v):
        if v >= 3:
            return a_ST[v-1,k-1]*model_W.W_STc[k,t] + b_ST[v-1,k-1]*model_W.s_STc[k,t] <= model_W.P_ST_pump[v-1,k,t]
        else:
            return Constraint.Skip
    model_W.WaterStoragePump_Constraint4_lower = Constraint(model_W.ST, model_W.T, model_W.V, rule=RegWaterPump_Constraint4_lower_rule)
    def WaterStoragePump_Constraint4_upper_rule(model_W, k, t, v):
        if v >= 3:
            return model_W.P_ST_pump[v-1,k,t] <= a_ST[v-1,k-1]*model_W.W_STc[k,t] + b_ST[v-1,k-1]*model_W.s_STc[k,t] + (1 - model_W.alpha_ST[v-1,k,t])*BigM
        else:
            return Constraint.Skip
    model_W.WaterStoragePump_Constraint4_upper = Constraint(model_W.ST, model_W.T, model_W.V, rule=WaterStoragePump_Constraint4_upper_rule)
    
    # MWM Optimization Solution
    Results_W_OBADMM = opt_W.solve(model_W)
    OBJ_W_OBADMM[k+1] = value(model_W.OBJ)
    for t in range(W_LOAD.size):
        P_water_W_OBADMM[k+1,t] = value(model_W.P_water_W_var[t+1])
        
    
    # Global Objective Value
    for t in range(P_LOAD.size):
        OBJ_global_OBADMM[k+1] = OBJ_E_OBADMM[k+1] + OBJ_W_OBADMM[k+1]
        
    # Lagrange Multiplier and Residuals Update
    for t in range(P_LOAD.size):
        lam_OBADMM[k+1,t] = lam_OBADMM[k,t] + rho*(P_water_E_OBADMM[k+1,t] - P_water_W_OBADMM[k+1,t]) # Updating Lagrange multiplier
        r_OBADMM[k+1,t] = P_water_E_OBADMM[k+1,t] - P_water_W_OBADMM[k+1,t] # Updating primal residual
        s_OBADMM[k+1,t] = (P_water_E_OBADMM[k+1,t] - P_water_W_OBADMM[k+1,t]) - (P_water_E_OBADMM[k,t] - P_water_W_OBADMM[k,t]) # Updating dual residual
    eps_OBADMM[k+1] = sqrt(np.sum(r_OBADMM[k+1,:])**2 + np.sum(s_OBADMM[k+1,:])**2) # Calculating feasibility quality metric
    
    k += 1 # Next iteration
    
    # Objective-Based Convergence Check
    if k > k_s: # Iterations past iteration offset number ?
        D_obj = np.array([])
        D_eps = np.array([])
        for a in range(k, k-(k_s),-1):
            D_obj = np.append(D_obj,abs(OBJ_global_OBADMM[a] - OBJ_global_OBADMM[a-1]))
            D_eps = np.append(D_eps,eps_OBADMM[a])
        if np.mean(D_obj) < beta and eps_OBADMM[k] < np.mean(D_eps): # Avg. Obj. Val. under preset threshold ? And Solution feasibility under avg. solution feasibility ?
            Check = 0
            
    if k >= 10000: # Maximum iterations
        break
        
    # Allocating space for next iteration
    if Check > 0:
        P_water_E_OBADMM = np.vstack((P_water_E_OBADMM, np.zeros(P_LOAD.size)))
        P_water_W_OBADMM = np.vstack((P_water_W_OBADMM, np.zeros(W_LOAD.size)))
        lam_OBADMM = np.vstack((lam_OBADMM, np.zeros(P_LOAD.size)))
        r_OBADMM = np.vstack((r_OBADMM, np.ones(P_LOAD.size)))
        s_OBADMM = np.vstack((s_OBADMM, np.ones(P_LOAD.size)))
        eps_OBADMM = np.append(eps_OBADMM,1)
        OBJ_E_OBADMM = np.append(OBJ_E_OBADMM,0)
        OBJ_W_OBADMM = np.append(OBJ_W_OBADMM,0)
        OBJ_global_OBADMM = np.append(OBJ_global_OBADMM,0)
    
# Main While Loop Ends

D_end_time = time.time()
D_COMP_TIME = D_end_time - D_start_time
if Check == 0:
    print(f"OB-ADMM Algorithm finished in {k} iterations.")
else:
    print(f"OB-ADMM Algorithm did no finish within {k} iterations.")
print("\nComputation time: {:.5f} sec.".format(D_COMP_TIME))
    

In [ ]:
print("Objective Value: {:.7f}".format(OBJ_global_OBADMM[-1]))
print("Normalized Obj. Value: {:.7f}".format(OBJ_global_OBADMM[-1]/value(model_CMWEN.OBJ)))
print("Final Solution Feasibility: {:.6e}".format(eps_OBADMM[-1]))

In [ ]:
# Total MWM Energy Consumption
print('Energy Consumption: {:0.3f} kWh'.format(sum(P_water_W_OBADMM[-1,t-1] for t in model_CMWEN.T)))

In [ ]:
# Total WMW Operation Power Consumption
plt.figure(figsize=(5, 4))
X = np.zeros(W_LOAD.size)
Y = np.zeros(W_LOAD.size)
for t in range(0, W_LOAD.size):
    X[t-1] = t+1
    Y[t-1] = value(P_water_W_OBADMM[-1,t])

plt.bar(X,Y,color='orangered')
plt.xticks(np.arange(2, max(X)+1, 2))
plt.xlabel('Hour')
plt.ylabel('Power [kWh]')
plt.title('Water Management Power Consumption')
plt.grid(True)
plt.gca().set_axisbelow(True)

In [ ]:
# Solution Feasibility Convergence
start = 4 # State initial iteration to look at
plt.figure(figsize=(5, 4))
X = np.array(range(start,k+1))
plt.plot(X,eps_OBADMM[start:k+1],color='darkblue')
plt.xlabel('Iteration (k)')
plt.ylabel('Solution Feasibility (Ɛ)')
plt.grid(True)
plt.gca().set_axisbelow(True)

In [ ]:
# Normalized Objective Value Convergence
start = 4 # State initial iteration to look at
plt.figure(figsize=(5, 4))
X = np.array(range(start,k+1))
plt.plot(X,OBJ_global_OBADMM[start:k+1]/value(model_CMWEN.OBJ),color='darkred')
plt.xlabel('Iteration (k)')
plt.ylabel('Normalized Obj. Val.')
plt.grid(True)
plt.gca().set_axisbelow(True)